__Boris meeting notes__

scan layers - 
* usually used on loops 
* more memorial 
* scanned - 
  * 1 more dimension 
  * vmap across 
  
where to add n-grammer: FlaxBartEncoder

need to edit partition: edit `_get_partition_rules` ('embedding')

examples to understand how to use pjit 
* google reserach /t5x
* huggingface transformers /examples/research porjects;https://github.com/huggingface/transformers/tree/main/examples/research_projects/jax-projects/model_parallel

the new base config
https://wandb.ai/dalle-mini/dalle-mini/runs/v0bq34cl/overview?workspace=user-borisd13



__command__
* baseline

python3 /home/yixu/dalle-yiyi/tools/train/train.py --assert_TPU_available  --do_train --do_eval --config_name config/mini_glu --dtype bfloat16 --tokenizer_name boris/dalle-mini-tokenizer --dataset_repo_or_path boris/gis_filtered --use_auth_token --blank_caption_prob 0.2 --output_dir output --overwrite_output_dir --per_device_train_batch_size 34 --gradient_accumulation_steps 3 --optim distributed_shampoo --block_size 1024 --beta1 0.9 --beta2 0.99 --learning_rate 0.0001 --warmup_steps 2000 --mp_devices 1 --logging_steps 100 --eval_steps 400 --save_steps 4000


* testing mp_devices=2

python3 /home/yixu/dalle-yiyi/tools/train/train.py --assert_TPU_available  --do_train --do_eval --config_name config/mini_glu --dtype bfloat16 --tokenizer_name boris/dalle-mini-tokenizer --dataset_repo_or_path boris/gis_filtered --use_auth_token --blank_caption_prob 0.2 --output_dir output --overwrite_output_dir --per_device_train_batch_size 34 --gradient_accumulation_steps 3 --optim distributed_shampoo --block_size 1024 --beta1 0.9 --beta2 0.99 --learning_rate 0.0001 --warmup_steps 2000 --mp_devices 2 --logging_steps 100 --eval_steps 400 --save_steps 4000

__hugggingface token__: https://huggingface.co/settings/tokens
* method1: 
```python
pip install huggingface_hub
huggingface-cli login
```
* method2:
```python
pip install huggingface_hub
python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token('MY_HUGGINGFACE_TOKEN_HERE')"
```



# training loop

train.py -main

In [2]:
pwd

### import

In [5]:
from pathlib import Path
root = Path('tools/train/')

In [6]:
# don't forget to do this next time (create 8 devices on cpu)
import os
os.environ['XLA_FLAGS'] = '--xla_force_host_platform_device_count=8'

In [7]:
import logging
import os
import sys
import tempfile
import time
from dataclasses import asdict, dataclass, field
from functools import partial
from typing import Any, Callable, NamedTuple, Optional

In [8]:
from flax.core.frozen_dict import FrozenDict, unfreeze
from flax.traverse_util import flatten_dict, unflatten_dict

In [378]:
import flax.linen as nn

In [410]:
import datasets
import flax
import jax
import jax.numpy as jnp
import jaxlib
import numpy as np
import optax
import transformers
import wandb
from datasets import Dataset
from flax import core, struct, traverse_util
from flax.core.frozen_dict import FrozenDict, freeze, unfreeze
from flax.serialization import from_bytes, to_bytes
from flax.training.common_utils import onehot
from jax.experimental import PartitionSpec, maps
from jax.experimental.compilation_cache import compilation_cache as cc
from jax.experimental.pjit import pjit, with_sharding_constraint
from tools.train.scalable_shampoo.distributed_shampoo import GraftingType, distributed_shampoo
from tqdm import tqdm
from transformers import HfArgumentParser

import dalle_mini
from dalle_mini.data import Dataset
from dalle_mini.model import (
    DalleBart,
    DalleBartConfig,
    DalleBartTokenizer,
    set_partitions,
)

try:
    from google.cloud import storage
except:
    storage = None

logger = logging.getLogger(__name__)


ImportError: cannot import name 'FlaxBartEncoder' from 'dalle_mini.model' (/Volumes/GoogleDrive/My Drive/Github/dalle-mini/src/dalle_mini/model/__init__.py)

In [ ]:
cc.initialize_cache("jax_cache")

### ModelArguments,DataTrainingArguments, TrainingArguments

In [10]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune, or train from scratch.
    """

    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The model checkpoint for weights initialization. "
            "Don't set if you want to train a model from scratch. "
            "W&B artifact references are supported in addition to the sources supported by `PreTrainedModel`."
        },
    )
    config_name: Optional[str] = field(
        default=None,
        metadata={
            "help": "Pretrained config name or path if not the same as model_name_or_path"
        },
    )
    tokenizer_name: Optional[str] = field(
        default=None,
        metadata={
            "help": "Pretrained tokenizer name or path if not the same as model_name_or_path"
        },
    )
    dtype: Optional[str] = field(
        default="float32",
        metadata={
            "help": "Floating-point format in which the computations will be performed (not the model weights). Choose one of `[float32, float16, bfloat16]`."
        },
    )
    restore_state: Optional[bool] = field(
        default=False,
        metadata={
            "help": "Restore optimizer and training state. Can be True (will retrieve associated wandb artifact), a local directory or a Google bucket path."
        },
    )
    dropout: Optional[float] = field(
        default=None,
        metadata={"help": "Dropout rate. Overwrites config."},
    )
    activation_dropout: Optional[float] = field(
        default=None,
        metadata={"help": "Activation dropout rate. Overwrites config."},
    )
    attention_dropout: Optional[float] = field(
        default=None,
        metadata={"help": "Attention dropout rate. Overwrites config."},
    )

    def __post_init__(self):
        if self.tokenizer_name is None:
            self.tokenizer_name = self.model_name_or_path
            assert (
                self.tokenizer_name is not None
            ), "Tokenizer name or model name/path needs to be specified"
        if self.restore_state:
            assert self.model_name_or_path is not None and (
                "/model-" in self.model_name_or_path
            ), "Restoring state only available with W&B artifact reference"

    def get_metadata(self):
        if self.model_name_or_path is not None and ":" in self.model_name_or_path:
            if jax.process_index() == 0:
                artifact = wandb.run.use_artifact(self.model_name_or_path)
            else:
                artifact = wandb.Api().artifact(self.model_name_or_path)
            return artifact.metadata
        else:
            return dict()

    def get_opt_state(self):
        with tempfile.TemporaryDirectory() as tmp_dir:  # avoid multiple artifact copies
            if self.restore_state is True:
                # wandb artifact
                state_artifact = self.model_name_or_path.replace(
                    "/model-", "/state-", 1
                )
                if jax.process_index() == 0:
                    artifact = wandb.run.use_artifact(state_artifact)
                else:
                    artifact = wandb.Api().artifact(state_artifact)
                if artifact.metadata.get("bucket_path"):
                    # we will read directly file contents
                    self.restore_state = artifact.metadata["bucket_path"]
                else:
                    artifact_dir = artifact.download(tmp_dir)
                    self.restore_state = str(Path(artifact_dir) / "opt_state.msgpack")

            if self.restore_state.startswith("gs://"):
                bucket_path = Path(self.restore_state[5:]) / "opt_state.msgpack"
                bucket, blob_name = str(bucket_path).split("/", 1)
                assert (
                    storage is not None
                ), 'Could not find google.storage. Install with "pip install google-cloud-storage"'
                client = storage.Client()
                bucket = client.bucket(bucket)
                blob = bucket.blob(blob_name)
                return blob.download_as_bytes()

            with Path(self.restore_state).open("rb") as f:
                return f.read()


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    text_column: Optional[str] = field(
        default="caption",
        metadata={
            "help": "The name of the column in the datasets containing the full texts (for summarization)."
        },
    )
    encoding_column: Optional[str] = field(
        default="encoding",
        metadata={
            "help": "The name of the column in the datasets containing the image encodings."
        },
    )
    dataset_repo_or_path: str = field(
        default=None,
        metadata={"help": "The dataset repository containing encoded files."},
    )
    train_file: Optional[str] = field(
        default=None,
        metadata={
            "help": "The input training data file (glob & braceexpand acceptable)."
        },
    )
    validation_file: Optional[str] = field(
        default=None,
        metadata={
            "help": "An optional input evaluation data file (glob & braceexpand acceptable)."
        },
    )
    # data loading should not be a bottleneck so we use "streaming" mode by default
    streaming: Optional[bool] = field(
        default=True,
        metadata={"help": "Whether to stream the dataset."},
    )
    use_auth_token: Optional[bool] = field(
        default=False,
        metadata={
            "help": "Whether to use the authentication token for private datasets."
        },
    )
    shard_by_host: Optional[bool] = field(
        default=False,
        metadata={
            "help": "Whether to shard data files by host in multi-host environments."
        },
    )
    blank_caption_prob: Optional[float] = field(
        default=0.0,
        metadata={
            "help": "Probability of removing some captions for classifier-free guidance."
        },
    )
    clip_score_column: Optional[str] = field(
        default="clip_score",
        metadata={"help": "Column that containts clip score for filtering."},
    )
    min_clip_score: Optional[float] = field(
        default=None,
        metadata={"help": "Minimum clip score required."},
    )
    max_clip_score: Optional[float] = field(
        default=None,
        metadata={"help": "Maximum clip score required."},
    )
    filter_column: Optional[str] = field(
        default=None,
        metadata={"help": "Column that containts classes to be filtered."},
    )
    filter_value: Optional[str] = field(
        default=None,
        metadata={"help": "Class value to be kept during filtering."},
    )
    multi_eval_ds: Optional[bool] = field(
        default=False,
        metadata={
            "help": "Whether to look for multiple validation datasets (local support only)."
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples."
        },
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={
            "help": "The number of processes to use for the preprocessing. Not used in streaming mode."
        },
    )
    overwrite_cache: bool = field(
        default=False,
        metadata={
            "help": "Overwrite the cached training and evaluation sets. Not used in streaming mode."
        },
    )
    # default seed of None ensures we don't repeat the same items if script was interrupted during an epoch
    seed_dataset: int = field(
        default=None,
        metadata={
            "help": "Random seed for the dataset that will be set at the beginning of training."
        },
    )

    def __post_init__(self):
        if self.dataset_repo_or_path is None:
            raise ValueError("Need a dataset repository or path.")


@dataclass
class TrainingArguments:
    """
    Arguments pertaining to training parameters.
    """

    output_dir: str = field(
        metadata={
            "help": "The output directory where the model predictions and checkpoints will be written."
        },
    )
    overwrite_output_dir: bool = field(
        default=False,
        metadata={
            "help": (
                "Overwrite the content of the output directory. "
                "Use this to continue training if output_dir points to a checkpoint directory."
            )
        },
    )

    do_train: bool = field(default=False, metadata={"help": "Whether to run training."})
    do_eval: bool = field(
        default=False, metadata={"help": "Whether to run eval on the validation set."}
    )

    per_device_train_batch_size: int = field(
        default=8,
        metadata={"help": "Batch size per data parallel device for training."},
    )
    per_device_eval_batch_size: Optional[int] = field(
        default=None,
        metadata={
            "help": "Batch size per data parallel device for evaluation. Same as training batch size if not set."
        },
    )

    gradient_accumulation_steps: int = field(
        default=1,
        metadata={
            "help": "Number of updates steps to accumulate before performing an update pass."
        },
    )
    gradient_checkpointing: bool = field(
        default=False, metadata={"help": "Use gradient checkpointing."}
    )

    learning_rate: float = field(
        default=5e-5, metadata={"help": "The initial learning rate."}
    )
    optim: str = field(
        default="distributed_shampoo",
        metadata={
            "help": 'The optimizer to use. Can be "distributed_shampoo" (default), "adam" or "adafactor"'
        },
    )
    weight_decay: float = field(
        default=0.0, metadata={"help": "Weight decay applied to parameters."}
    )
    beta1: float = field(
        default=0.9,
        metadata={"help": "Beta1 for Adam & Distributed Shampoo."},
    )
    beta2: float = field(
        default=0.999,
        metadata={"help": "Beta2 for for Adam & Distributed Shampoo."},
    )
    adam_epsilon: float = field(
        default=1e-8, metadata={"help": "Epsilon for AdamW optimizer."}
    )
    max_grad_norm: float = field(
        default=1.0, metadata={"help": "Max gradient norm for Adafactor."}
    )
    block_size: int = field(
        default=1024,
        metadata={"help": "Chunked size for large layers with Distributed Shampoo."},
    )
    preconditioning_compute_steps: int = field(
        default=10, metadata={"help": "Number of steps to update preconditioner."}
    )
    skip_preconditioning_dim_size_gt: int = field(
        default=4096,
        metadata={"help": "Max size for preconditioning with Distributed Shampoo."},
    )
    graft_type: str = field(
        default="rmsprop_normalized",
        metadata={
            "help": "The type of grafting to use. Can be 'rmsprop_normalized' (default), 'rmsprop', 'adagrad', 'adagrad_normalized', 'sgd' or 'sqrt_n'"
        },
    )
    nesterov: bool = field(
        default=False,
        metadata={"help": "Use Nesterov momentum for Distributed Shampoo."},
    )
    optim_quantized: bool = field(
        default=False,
        metadata={
            "help": "Whether to quantize optimizer (only supported with Distributed Shampoo)."
        },
    )
    shard_shampoo_across: str = field(
        default="dp",
        metadata={
            "help": "Whether to shard the optimizer across data devices (dp), model devices (mp) or both (2d)."
        },
    )

    num_train_epochs: int = field(
        default=3, metadata={"help": "Total number of training epochs to perform."}
    )

    warmup_steps: int = field(
        default=0, metadata={"help": "Linear warmup over warmup_steps."}
    )
    lr_decay: str = field(
        default=None,
        metadata={
            "help": "Decay to be used in the learning rate scheduler. Can be None (default), linear or exponential."
        },
    )
    lr_transition_steps: int = field(
        default=None,
        metadata={
            "help": "Number of transition steps associated with learning rate decay when using exponential decay."
        },
    )
    lr_decay_rate: float = field(
        default=None,
        metadata={
            "help": "Decay rate associated with learning rate when using exponential decay."
        },
    )
    lr_staircase: bool = field(
        default=False,
        metadata={
            "help": "Whether to use staircase or continuous learning rate when using exponential decay."
        },
    )
    lr_offset: int = field(
        default=0,
        metadata={"help": "Number of steps to offset learning rate and keep it at 0."},
    )
    logging_steps: int = field(
        default=40, metadata={"help": "Log every X updates steps."}
    )
    eval_steps: int = field(
        default=400, metadata={"help": "Run an evaluation every X steps."}
    )
    save_steps: int = field(
        default=4000, metadata={"help": "Save checkpoint every X updates steps."}
    )
    log_model: bool = field(
        default=False,
        metadata={"help": "Log model to wandb at `save_steps` frequency."},
    )
    log_norm_steps: int = field(
        default=True,
        metadata={"help": "Log parameters and gradients norm at this frequency."},
    )
    log_histogram_steps: int = field(
        default=False,
        metadata={
            "help": "Log parameters and gradients histograms at this frequency. Slows down training."
        },
    )

    seed_model: int = field(
        default=42,
        metadata={
            "help": "Random seed for the model that will be set at the beginning of training."
        },
    )

    embeddings_only: bool = field(
        default=False, metadata={"help": "Train only embedding layers."}
    )
    init_embeddings: bool = field(
        default=False,
        metadata={"help": "When training embedding layers, initialize them."},
    )

    wandb_entity: Optional[str] = field(
        default=None,
        metadata={"help": "The wandb entity to use (for teams)."},
    )
    wandb_project: str = field(
        default="dalle-mini",
        metadata={"help": "The name of the wandb project."},
    )
    wandb_job_type: str = field(
        default="Seq2Seq",
        metadata={"help": "The name of the wandb job type."},
    )

    assert_TPU_available: bool = field(
        default=False,
        metadata={"help": "Verify that TPU is not in use."},
    )

    use_vmap_trick: bool = field(
        default=True,
        metadata={"help": "Verify that TPU is not in use."},
    )

    mp_devices: Optional[int] = field(
        default=1,
        metadata={
            "help": "Number of devices required for model parallelism. The other dimension of available devices is used for data parallelism."
        },
    )

    dp_devices: int = field(init=False)

    def __post_init__(self):
        if self.assert_TPU_available:
            assert (
                jax.local_device_count() == 8
            ), "TPUs in use, please check running processes"
        if self.output_dir.startswith("gs://"):
            assert (
                storage is not None
            ), 'Could not find google.storage. Install with "pip install google-cloud-storage"'
        assert self.optim in [
            "distributed_shampoo",
            "adam",
            "adafactor",
        ], f"Selected optimizer not supported: {self.optim}"
        if self.optim == "adafactor" and self.weight_decay == 0:
            self.weight_decay = None
        assert self.graft_type in [
            "rmsprop_normalized",
            "rmsprop",
            "adagrad",
            "adagrad_normalized",
            "sgd",
            "sqrt_n",
        ], f"Selected graft type not supported: {self.graft_type}"
        assert self.lr_decay in [
            None,
            "linear",
            "exponential",
        ], f"Selected learning rate decay not supported: {self.lr_decay}"
        if self.per_device_eval_batch_size is None:
            self.per_device_eval_batch_size = self.per_device_train_batch_size
        if self.log_norm_steps is True:
            self.log_norm_steps = self.logging_steps
        if not self.do_train:
            self.num_train_epochs = 1
        if (
            os.path.exists(self.output_dir)
            and os.listdir(self.output_dir)
            and self.do_train
            and not self.overwrite_output_dir
        ):
            raise ValueError(
                f"Output directory ({self.output_dir}) already exists and is not empty."
                "Use --overwrite_output_dir to overcome."
            )
        assert self.shard_shampoo_across in [
            "dp",
            "mp",
            "2d",
        ], f"Shard shampoo across {self.shard_shampoo_across} not supported."
        assert (
            self.mp_devices > 0
        ), f"Number of devices for model parallelism must be > 0"
        assert (
            jax.device_count() % self.mp_devices == 0
        ), f"Number of available devices ({jax.device_count()} must be divisible by number of devices used for model parallelism ({self.mp_devices})."
        self.dp_devices = jax.device_count() // self.mp_devices


### create parser

In [11]:
parser = HfArgumentParser(
        (ModelArguments, DataTrainingArguments, TrainingArguments)
    )

based on __base config__:


https://wandb.ai/dalle-mini/dalle-mini/runs/mheh9e55/overview?workspace=user-borisd13

In [12]:
# based on https://wandb.ai/dalle-mini/dalle-mini/runs/mheh9e55/overview?workspace=user-borisd13
# I removed --assert_TPU_available --wandb_entity dalle-mini
# make mp_device = 1 
  # tried to make mp_device=1 but it doesn't work (not dividable)
commandline = ["--do_train", "--do_eval",
               "--config_name", "config/mini_glu",
                          "--dtype", "bfloat16",
                          "--tokenizer_name", "boris/dalle-mini-tokenizer",
                          "--dataset_repo_or_path", "boris/gis_filtered",
                          "--use_auth_token",
                          "--blank_caption_prob", "0.2",
                          "--output_dir", "output",
                          "--overwrite_output_dir",
                          "--per_device_train_batch_size", "34",
                          "--gradient_accumulation_steps", "3",
                          "--optim", "distributed_shampoo",
                          "--block_size", "1024",
                          "--beta1", "0.9",
                          "--beta2", "0.99",
                          "--learning_rate", "0.0001",
                          "--warmup_steps", "2000",
                          "--mp_devices", "1",
                          "--logging_steps", "100",
                          "--eval_steps", "400",
                          "--save_steps", "4000",
                          "--log_model"]

### parse the arguments into 3 dicts: model_args, data_args and training_args

In [13]:
model_args, data_args, training_args = parser.parse_args_into_dataclasses(commandline)

In [14]:
model_args

ModelArguments(model_name_or_path=None, config_name='config/mini_glu', tokenizer_name='boris/dalle-mini-tokenizer', dtype='bfloat16', restore_state=False, dropout=None, activation_dropout=None, attention_dropout=None)

In [15]:
training_args

TrainingArguments(output_dir='output', overwrite_output_dir=True, do_train=True, do_eval=True, per_device_train_batch_size=34, per_device_eval_batch_size=34, gradient_accumulation_steps=3, gradient_checkpointing=False, learning_rate=0.0001, optim='distributed_shampoo', weight_decay=0.0, beta1=0.9, beta2=0.99, adam_epsilon=1e-08, max_grad_norm=1.0, block_size=1024, preconditioning_compute_steps=10, skip_preconditioning_dim_size_gt=4096, graft_type='rmsprop_normalized', nesterov=False, optim_quantized=False, shard_shampoo_across='dp', num_train_epochs=3, warmup_steps=2000, lr_decay=None, lr_transition_steps=None, lr_decay_rate=None, lr_staircase=False, lr_offset=0, logging_steps=100, eval_steps=400, save_steps=4000, log_model=True, log_norm_steps=100, log_histogram_steps=False, seed_model=42, embeddings_only=False, init_embeddings=False, wandb_entity=None, wandb_project='dalle-mini', wandb_job_type='Seq2Seq', assert_TPU_available=False, use_vmap_trick=True, mp_devices=1, dp_devices=8)

### create dataset

In [16]:
from datasets import load_dataset as hf_load_dataset

In [17]:
dataset = Dataset(
        **asdict(data_args),
        do_train=training_args.do_train,
        do_eval=training_args.do_eval,
    )

In [18]:
print(f"Local TPUs: {jax.local_device_count()}")
logger.info(f"Global TPUs: {jax.device_count()}")

Local TPUs: 8


In [19]:
logger

<Logger __main__ (WARNING)>

### get model config

In [20]:
#  look for dropout and gradient_checkpointing
# add to the default config if any
config_args = {
        k: getattr(model_args, k)
        for k in ["dropout", "activation_dropout", "attention_dropout"]
        if getattr(model_args, k) is not None
    }
config_args["gradient_checkpointing"] = training_args.gradient_checkpointing

In [21]:
config_args

{'gradient_checkpointing': False}

In [22]:
# create config as DalleBartConfig
config = DalleBartConfig.from_pretrained(str(root /model_args.config_name))

In [23]:
config

DalleBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "attention_dropout": 0.0,
  "bos_token_id": 16385,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 2730,
  "decoder_layers": 12,
  "decoder_start_token_id": 16384,
  "do_sample": true,
  "dropout": 0.0,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 2730,
  "encoder_layers": 12,
  "encoder_vocab_size": 50300,
  "eos_token_id": 16385,
  "force_ln_scale": false,
  "gradient_checkpointing": false,
  "image_length": 256,
  "image_vocab_size": 16400,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "ln_positions": "normformer",
  "ln_type": "layernorm",
  "max_length": 257,
  "max_text_length": 64,
  "min_length": 257,
  "model_type": "dallebart",
  "normalize_text": true,
  "pad_token_id": 16385,
  "scale_embedding": false,
  "sinkhorn_iters": 1,
  "tau_init": 0.05,
  "tie_word_embeddings": false,
  "transformers_version": "4.24.0.dev0",
  "use_absolute_position_embeddings":

### create model

In [24]:
# did not provide a model path so create a new model 
model_args.model_name_or_path

In [25]:
training_args.seed_model

42

In [26]:
# dtype in tpu is bfloat16
model_args.dtype

'bfloat16'

In [27]:
getattr(jnp, model_args.dtype)

jax.numpy.bfloat16

In [28]:
jnp.bfloat16

jax.numpy.bfloat16

In [29]:
# create model
model = DalleBart(
            config,
            seed=training_args.seed_model,
            dtype=getattr(jnp, model_args.dtype),
            _do_init=False,
        )

In [30]:
params = None

In [31]:
# add the config_args (dropout and gradient_checkpointing) to model config
for k, v in config_args.items():
    setattr(model.config, k, v)

In [32]:
id(model.config), id(config)

(5180626384, 5180626384)

In [33]:
# get model metadata
model_metadata = model_args.get_metadata()

In [34]:
model_metadata

{}

### !! get partitionspec -> param_spec

use __set_partitions__ function to get __ParitionSpec__ for params 

```python
param_spec = set_partitions(params_shape, model.config.use_scan)
```
* input __params_shape__ is a tree contains shapes info(i.e. __ShapeDtypeStruct__) for each parameters
* returns __param_spec__ is a tree with same structure but with __PartitionSpec__

what is __PartitionSpec__?
* __resource assignment specification__, used to define __pjit__'s argument __in_axis_resources__ and __out_axis_resources__ : 
```python
pjit(fn, in_axis_resources= .., out_axis_resources=..)(data)
```
* it is a __tuple__ of __length__ at most equal to the __rank of the partitioned value__ 
  * partitioned value?
    * in the example above, __data__ and __in_axis_resource__ should both be pytree with __same structure__, one contain __partitioned value__, one contains the __PartitionSpec__, each PartitionedSpec has a corresponding paritioned value in __data__
* <u>each element in PartitionSpec</u> can be a __None__, a __mesh axis__ or a __tuple of mesh axes__, and specifies the set of resources assigned to partition the <u>value’s dimension</u> matching its position in the spec
* we will look at our example soon


In [35]:
# arguments
model.config.use_scan

False

In [36]:
# params_shape_tree is a property of FlaxPreTrainedModel - it's created when the model object is created
params_shape = model.params_shape_tree

In [37]:
 # get PartitionSpec for model params (required to be a dict)
param_spec = set_partitions(params_shape, model.config.use_scan)

In [38]:
params_shape = freeze(params_shape)

How is __param_spec__ used in dalle-mini?

* assume we load the model from a checkpoint(which we are not now) , we will need to use param_spec to create __initial train state__
* our __params__ is None right now, but assume we are loading model from a checkpoint, the params variable will be a pytree with same structure as __param_spec__, and you can checkout the shape for each individual parameter in __params_shape__ below

```python
state = pjit(
                init_state,
                in_axis_resources=(param_spec,)
                if model_args.model_name_or_path
                else None,
                out_axis_resources=state_spec,
                donate_argnums=(0,),
            )(params)
```

below, we will taking one __parameter__ (i.e. partitioned value) as example, trying to understand its __PartitionSpec__ 

* the parameter __P__ has 2 dimensions: __<font color=red>p[0]</font>__, __P[1]__ (1024 x 16385)
    ```python
    p(0,0),    p(0,1), ..., p(0,16384)
    p(1,0),    p(1,1), ..., p(1,16384)
    
    ...
    p(1023,0), p(1023,1),...,p(1023,16384)
    ```
* the __mesh__ has 2 dimensions: __'dp'__, __'mp'__ (4 x 2) 
    ```python
    d0, d1, 
    d2, d3
    d4, d5, 
    d6, d7
    ```
* the PartitionSpec __PartitionSpec(<font color=red>None</font>, 'mp')__ has 2 elements, corresponding to the __<font color=red>p[0]</font>__ and __p[1]__. and it says we want to <font color=blue>shard</font> __p[1]__ across the mesh axis __'mp'__ and <font color=blue>duplicated</font> __<font color=red>p[0]</font>__ across mesh axis <font color=red>__'dp'__</font>
 
 
so the 4 devices on the first column __(i.e. d0,d2,d4,d6)__ will have same slice of data 
  ```python
   p(0,0),    p(0,1),  ..., p(0,8192)
   p(1,0),    p(1,1),  ..., p(1,8192)
                 ...
   p(1023,0), p(1023,1),...,p(1023,8192)
   ```
    
the 4 devices on the second column __(d1,d3,d5,d7)__ will also have same slice of data

   ```python
   p(0,8192),    p(0,8193),  ...,    p(0,16384)
   p(1,8192),    p(1,8193),  ...,    p(1,16384)
                 ...
   p(1023,8192), p(1023,8193),  ..., p(1023,16384)
   ```
  
 we will validate this later

In [39]:
# take one parameter as example, i.e. lm_head /kernel, denote as P
# P has 2 dimensions, size 1024 x 16385 
params_shape['lm_head']['kernel']

ShapeDtypeStruct(shape=(1024, 16401), dtype=float32)

In [40]:
#P's partitionspec in param_spec
# None <-> p[0] <-> mesh axis 'dp' : the first dimension (size 1024 will be replicated across dp =4)
# 'mp' -> p[1] <-> mesh axis 'mp': the second dimension (size 16385 will be sharded across mp = 2)
param_spec['lm_head']['kernel']

PartitionSpec(None, 'mp')

PartitionSpec for parameters
* it seems like embedding dimension (1024) is always None
* we want to make sure all device has access to all embedding dimensions(?)

In [239]:
param_spec

FrozenDict({
    lm_head: {
        kernel: PartitionSpec(None, 'mp'),
    },
    model: {
        decoder: {
            embed_positions: {
                embedding: PartitionSpec('mp', None),
            },
            embed_tokens: {
                embedding: PartitionSpec('mp', None),
            },
            final_ln: {
                bias: None,
            },
            layernorm_embedding: {
                bias: None,
                scale: None,
            },
            layers: {
                FlaxBartDecoderLayer_0: {
                    FlaxBartAttention_0: {
                        k_proj: {
                            kernel: PartitionSpec(None, 'mp'),
                        },
                        out_proj: {
                            kernel: PartitionSpec('mp', None),
                        },
                        q_proj: {
                            kernel: PartitionSpec(None, 'mp'),
                        },
                        v_proj: {
    

In [240]:
params_shape

FrozenDict({
    lm_head: {
        kernel: ShapeDtypeStruct(shape=(1024, 16401), dtype=float32),
    },
    model: {
        decoder: {
            embed_positions: {
                embedding: ShapeDtypeStruct(shape=(256, 1024), dtype=float32),
            },
            embed_tokens: {
                embedding: ShapeDtypeStruct(shape=(16401, 1024), dtype=float32),
            },
            final_ln: {
                bias: ShapeDtypeStruct(shape=(1024,), dtype=float32),
            },
            layernorm_embedding: {
                bias: ShapeDtypeStruct(shape=(1024,), dtype=float32),
                scale: ShapeDtypeStruct(shape=(1024,), dtype=float32),
            },
            layers: {
                FlaxBartDecoderLayer_0: {
                    FlaxBartAttention_0: {
                        k_proj: {
                            kernel: ShapeDtypeStruct(shape=(1024, 1024), dtype=float32),
                        },
                        out_proj: {
                     

### load tokenizer

In [41]:
tokenizer = DalleBartTokenizer.from_pretrained(
        model_args.tokenizer_name, use_fast=True
    )

In [361]:
tokenizer

PreTrainedTokenizerFast(name_or_path='boris/dalle-mini-tokenizer', vocab_size=50265, model_max_len=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

In [367]:
tokenizer.vocab['<s>']

0

In [368]:
tokenizer.vocab['</s>']

2

In [369]:
tokenizer.vocab['<unk>']

3

In [371]:
tokenizer.vocab['<pad>']

1

In [362]:
len(tokenizer)

50265

### preprocessing dataset
* filter dataset - doesn't do anything with our base config
* apply a normalize function on captions -> not sure what it does 
* randomly replace 20% of captions as blank texts
* create output
  * input_ids: tokenized caption
  * attention_mask: output of tokenizer as well 
  * "labels" (i.e. example["encoding"])
    *  __what is example['encoding']?__ output from vqgan encoder 
  * "decoder_input_ids" (same as labels but shift right 1 position, adding bos token)
    * e.g. tok1, tok2, tok3 -> bos, tok1, tok2

In [44]:
# Preprocessing the datasets.
# We need to normalize and tokenize inputs and targets.
dataset.preprocess(tokenizer=tokenizer, config=model.config)

In [45]:
# vqgan encode images to 16 x 16 = 256 discrete tokens from a vocabulary of size 16384,
len(next(iter(dataset.train_dataset)['encoding']))

TypeError: 'generator' object is not subscriptable

In [46]:
example_preprocessed = next(iter(dataset.train_dataset))

In [47]:
example_preprocessed

{'input_ids': array([    0,    75,  1455,    31,    11,  7315, 16612, 25641,  5072,
         7406,    11,   176, 49895,   107,   820,   125, 10795,     2,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1]),
 'attention_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': array([ 1304, 15525, 14164,  7191,  8173,  1495, 16147,  8173,  2428,
         8173, 14447,  8173, 14164, 14164,  1755, 11605,  4815,  2528,
         2528,  6965,  3212, 11674,  9157,  3547,  4236,  5900,  8493,
        14447,

In [49]:
len(example_preprocessed['labels'])

256

In [50]:
example_preprocessed['labels'].max()

16147

In [521]:
jax.tree_map(lambda x:x.shape, example_preprocessed)

{'attention_mask': (64,),
 'decoder_input_ids': (256,),
 'input_ids': (64,),
 'labels': (256,)}

In [516]:
example_preprocessed.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

#### step by step

In [ ]:
# required config variables, not sure what they are at this time
decoder_start_token_id = config.decoder_start_token_id
normalize_text = config.normalize_text
max_length = config.max_text_length

In [ ]:
decoder_start_token_id

In [ ]:
normalize_text, max_length

In [ ]:
# our data looks like this, caption and its encoding
example = next(iter(dataset.train_dataset))

In [ ]:
example['caption']

In [ ]:
len(example['encoding'])

filter datasets - 
* filter for both `train_dataset` and `eval_dataset`, i will take train_dataset as example
* in our case, this step doesn't actually do anything 

In [ ]:
for example in dataset.train_dataset:
    if example['clip_score'] is not None:
        print(example['clip_score'])

In [ ]:
type(dataset.train_dataset), type(dataset.eval_dataset)

for our configuration, this `partial_filter_function` evaluate as 

```python
def filter_function(example):
  return True
```

In [ ]:
dataset.clip_score_column

In [ ]:
def filter_function(
    example,
    min_clip_score,
    max_clip_score,
    clip_score_column,
    filter_column,
    filter_value,
):
    if min_clip_score is not None and example[clip_score_column] < min_clip_score:
        return False
    if max_clip_score is not None and example[clip_score_column] > max_clip_score:
        return False
    if filter_column is not None and example[filter_column] != filter_value:
        return False
    return True


partial_filter_function = partial(
            filter_function,
            filter_column=None,
            filter_value=None,
            clip_score_column='clip_score',
            min_clip_score=None,
            max_clip_score=None,
        )

In [ ]:
# dataset.train_dataset.filter??

In [ ]:
dataset.train_dataset.filter(partial_filter_function)

In [ ]:
dataset.length

apply a normalize function to each caption

In [ ]:
dataset.text_column

In [ ]:
from src.dalle_mini.model.text import TextNormalizer

In [ ]:
text_normalizer=TextNormalizer()

In [ ]:
def normalize_function(example, text_column, text_normalizer):
    example[text_column] = text_normalizer(example[text_column])
    return example

`partial_normalize_function` evaluate as 
```python
def partial_normalize_function(example):
    return text_normalizer(example['caption'])
```
    


blank captions??

20% of the time, will set the caption to be blank - not sure why

In [ ]:
dataset.blank_caption_prob

In [ ]:
dataset.np_rng

In [ ]:
def blank_caption_function(example, text_column, blank_caption_prob, rng=None):
    if (
        blank_caption_prob
        and (rng.random() if rng is not None else np.random.random())
        < blank_caption_prob
    ):
        example[text_column] = ""
    return example

preprocess_function? 

partial_preprocess_function takes a batch of exampls as input and return a batch of outputs each contains: 
* label:  example['encoding']: tok1, tok2, tok3
* decoder_input_ids: bos, tok1, tok2

In [ ]:
dataset.text_column

In [ ]:
dataset.encoding_column

In [ ]:
max_length

In [ ]:
decoder_start_token_id

In [ ]:
def preprocess_function(
    examples,
    tokenizer, # passed from train.py
    text_column, # 'caption'
    encoding_column, #'encoding'
    max_length,#64
    decoder_start_token_id, # 16384
):
    inputs = examples[text_column] # caption
    # Setting padding="max_length" as we need fixed length inputs for jitted functions
    model_inputs = tokenizer(
        inputs,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="np",
    )

    # set up targets
    # Note: labels correspond to our target indices
    # decoder input ids are the same but shifted to the right with bos at the beginning (and without last token)
    labels = examples[encoding_column]
    labels = np.asarray(labels)

    # We need the labels, in addition to the decoder_input_ids, for the compute_loss function
    model_inputs["labels"] = labels

    # In our case, this prepends the bos token and removes the last one
    decoder_input_ids = shift_tokens_right(labels, decoder_start_token_id)
    model_inputs["decoder_input_ids"] = decoder_input_ids

    return model_inputs

In [ ]:
dataset.train_dataset.map??

### calculate training related variables (different batch sizes)

num_epochs, different batch sizes, steps_per_epoch, num_train_steps, num_params

In [51]:
dropout_rng = jax.random.PRNGKey(training_args.seed_model)

In [52]:
num_epochs = training_args.num_train_epochs
num_epochs

3

In [53]:
 # batch size : batch size (per device) * number of devices // node? 
 # jax.local_device_count() = number of TPU devices
batch_size_per_node_per_grad_step = (
        training_args.per_device_train_batch_size
        * jax.local_device_count()
        // training_args.mp_devices
    )

In [54]:
training_args.per_device_train_batch_size

34

In [55]:
training_args.mp_devices

1

In [56]:
batch_size_per_node_per_grad_step

272

In [57]:
training_args.gradient_accumulation_steps

3

In [58]:
# I guess we updated gradient every 3 steps, so the gradient update is calculated based on 3 batches
batch_size_per_node = (
        batch_size_per_node_per_grad_step * training_args.gradient_accumulation_steps
    )

In [59]:
batch_size_per_node

816

In [60]:
# jax.process_count = number of nodes? MP?
batch_size_per_step = batch_size_per_node * jax.process_count()

In [61]:
eval_batch_size_per_node = (
        training_args.per_device_eval_batch_size
        * jax.local_device_count()
        // training_args.mp_devices
    )

In [62]:
eval_batch_size_per_node

272

In [63]:
eval_batch_size_per_step = eval_batch_size_per_node * jax.process_count()

In [64]:
dataset.length

(None, None)

In [65]:
len_train_dataset, len_eval_dataset = dataset.length

In [66]:
 steps_per_epoch = (
        len_train_dataset // batch_size_per_node
        if len_train_dataset is not None
        else None
    )

In [67]:
# None is our case
steps_per_epoch

In [68]:
# also None is our case
num_train_steps = (
        steps_per_epoch * num_epochs if steps_per_epoch is not None else None
    )
num_train_steps

In [69]:
num_params = model.num_params(params_shape)

In [70]:
num_params

437903344

logging the ablove info info about model and data to both logging and wandb config with 

`wandb.config.update({...})`

### create a learning_rate_fn

* if lr_offset >0, say for example = 100, there will be 100 steps with 0 learnings before warmup - <font color=red>what is this for?</font>
* first 2000 steps linearly warm up ( lr from 0 ~ 0.0001), then constant at 0.0001 - this is our __default setting__
* optionally, you can set up exponential or linear decay function for the rest of your training steps 


In [71]:
training_args.learning_rate

0.0001

In [72]:
training_args.warmup_steps

2000

In [73]:
warmup_fn = optax.linear_schedule(
            init_value=0.0,
            end_value=training_args.learning_rate,
            transition_steps=training_args.warmup_steps + 1,  # ensure not 0
        )

In [74]:
training_args.lr_offset

0

In [75]:
#our num_train_steps is None, so it will have a warmup phase (2000) and then train with 1e-3 forever
num_train_steps is None

True

In [76]:
training_args.lr_decay is None

True

In [77]:
    def create_learning_rate_fn() -> Callable[[int], jnp.array]:
        """Create the learning rate function."""
        warmup_fn = optax.linear_schedule(
            init_value=0.0,
            end_value=training_args.learning_rate,
            transition_steps=training_args.warmup_steps + 1,  # ensure not 0
        )
        last_boundary = training_args.warmup_steps
        # offset step when resuming
        if training_args.lr_offset:
            warmup_fn = optax.join_schedules(
                schedules=[optax.constant_schedule(0.0), warmup_fn],
                boundaries=[training_args.lr_offset],
            )
            last_boundary += training_args.lr_offset
        if training_args.lr_decay is None:
            return warmup_fn
        elif training_args.lr_decay == "linear":
            assert (
                num_train_steps is not None
            ), "linear decay requires knowing the dataset length"
            decay_fn = optax.linear_schedule(
                init_value=training_args.learning_rate,
                end_value=0,
                transition_steps=num_train_steps - training_args.warmup_steps,
            )
        elif training_args.lr_decay == "exponential":
            decay_fn = optax.exponential_decay(
                init_value=training_args.learning_rate,
                transition_steps=training_args.lr_transition_steps,
                decay_rate=training_args.lr_decay_rate,
                staircase=training_args.lr_staircase,
            )
        schedule_fn = optax.join_schedules(
            schedules=[warmup_fn, decay_fn],
            boundaries=[last_boundary],
        )
        return schedule_fn


In [78]:
learning_rate_fn = create_learning_rate_fn()

In [79]:
# try a really large number of steps
learning_rate_fn(1000000)

DeviceArray(1.e-04, dtype=float32, weak_type=True)

### trainable_params_shape
* base config with embedding_only == False, all parameters are trainable : __trainable_params_shape__ is same as __params_shape__

In [80]:
def trainable_params(data, embeddings_only):
    """Keep only trainable parameters"""

    if not embeddings_only:
        return data

    data = unfreeze(data)
    trainable = {
        "lm_head": data["lm_head"],
        "model": {
            "decoder": {
                layer: data["model"]["decoder"][layer]
                for layer in [
                    "embed_positions",
                    "embed_tokens",
                    "final_ln",
                    "layernorm_embedding",
                ]
            }
        },
    }
    return freeze(trainable)

if set embedding_only = True, this part of parameters won't be updated

In [81]:
trainable_params_shape = trainable_params(
        params_shape, training_args.embeddings_only
    )

In [82]:
# actually exact same object
id(trainable_params_shape), id(params_shape)

(5177362800, 5177362800)

### read about pjit before create optimizer
https://jax.readthedocs.io/en/latest/jax-101/08-pjit.html

with pmap, we only partition data, i.e. same model are run on each device with a subset of data; but with pjit, we can use it to partition functions as well - we can partition models and have subsets of the model run in parallel across multiple devices 

parallel model training 
* model 
```python
        layer1   ➡️   layer 2 
     parameter1  ➡️   parameter2
      [1,2,3]       [4,5,6,7,8,9]

 device0:[1]              [4,5]
 device1:[2]              [6,7]
 device2:[3]              [8,9]
    
```
        
* to be clear, we don't partition the layers: all the devices will get all layers in the same sequence
* with pjit, we can partition the parameters, so each device will get a subset of model and these models can run in parallel 
* need to understand more about __how to partition the parameters__? for example, for attention layers, if we partition the sequence length dimension, some tokens will not be able to interact with others? partition the head dimension will probably be ok

__pjit__ is the API for for core infrastructure __XLA SPMD partitioner__
* SPMD: single program, multiple data?
* input: 
  * function 
  * partitioning specifications for function inputs - (`in_axis_resources`)
  * partitioning specification for outputs (`out_axis_resources`)

what is mesh?

the resources specified in these 2 arguments must refer to mesh axis, as defined by the jax jax.experimental.maps.Mesh() context manager.

in our case, our mesh axises are: 'dp', 'mp' - the first dimension is data parallelism, second dimension is model parallelism

* when we partition the parameters, the "dp" axis should always be None (see our param_spec below)

In [78]:
jax.tree_leaves(param_spec)

In [ ]:
from jax.experimental import maps

In [ ]:
# data parallelism, model parallelism
# pjit is normally used on tpu pod slice, i.e. multiple tpu next to each other (32 devices or so)
# but you can still use pjit with 1 tpu (8 devices), here we do 4x2
mesh_shape = (training_args.dp_devices, training_args.mp_devices)
devices = np.asarray(jax.devices()).reshape(*mesh_shape)
devices, devices.shape

In [ ]:
# mp: model parallelism
# it is specified in argument 
training_args.mp_devices

In [79]:
# dp: data parallelism: 
# it is calculated by number of total devices divided by mp
training_args.dp_devices

In [ ]:
jax.device_count() // training_args.mp_devices

In [ ]:
mesh = maps.Mesh(devices, ("dp", "mp"))

In [ ]:
mesh

Inputs to a pjitted function will be __automatically partitioned__ across __devices__ if they’re not already correctly partitioned based on __in_axis_resources.__

* in dalle-mini example, 
  * we pjitted the function to create the initial trainstate it is called inside train_step function
  * train_step is also pjitted 
```python
def init_state(params):
    return TrainState.create(...)

state = pjit(init_state,
             in_axis_resources=(param_spec,)
             None,
             out_axis_resources=state_spec,
             donate_argnums=(0,),)(params)
  
def train_step(state, batch, train_time):
    ...
    return state, metrics
                    
p_train_step = pjit(
    train_step,
    in_axis_resources=(state_spec,batch_spec,None,),
    out_axis_resources=(state_spec, None),
    donate_argnums=(0,),
    )

state, train_metrics = p_train_step(state, batch, train_time)
```

read more about buffer donation https://jax.readthedocs.io/en/latest/faq.html#buffer-donation


### create optimizer: distributed_shampoo -> opt, statistics_partition_spec

* __opt__ is a GradientTransformation object, just like other optimizers we pass as tx to TrainState; however, it is used differentl for pjit mode (more on this in next section)
* __statistics_partition_spec__ is the PartitionSpec for shampoo's statistics - not sure how it is used 

In [83]:
# shampoo is a second-order optimizer 
training_args.optim

'distributed_shampoo'

In [84]:
from tools.train.scalable_shampoo.distributed_shampoo import GraftingType, distributed_shampoo

In [85]:
GraftingType

<enum 'GraftingType'>

In [86]:
training_args.graft_type

'rmsprop_normalized'

In [87]:
# graft_type - not sure what this is 
graft_type = {
            "sgd": GraftingType.SGD,
            "adagrad": GraftingType.ADAGRAD,
            "rmsprop": GraftingType.RMSPROP,
            "rmsprop_normalized": GraftingType.RMSPROP_NORMALIZED,
            "sqrt_n": GraftingType.SQRT_N,
            "adagrad_normalized": GraftingType.ADAGRAD_NORMALIZED,
        }[training_args.graft_type]

In [88]:
graft_type

<GraftingType.RMSPROP_NORMALIZED: 4>

In [89]:
from jax.experimental import PartitionSpec, maps

In [90]:
# shard across dp - what does this mean?
training_args.shard_shampoo_across 

'dp'

In [91]:
# create statistics_partition_spec
# I'm not sure how it is used
statistics_partition_spec = (
            PartitionSpec(None, training_args.shard_shampoo_across, None)
            if training_args.shard_shampoo_across != "2d"
            else PartitionSpec(None, "dp", "mp")
        )
            

In [92]:
statistics_partition_spec

PartitionSpec(None, 'dp', None)

In [93]:
# arguments to create `distributed_shampoo`
opt = distributed_shampoo(
            learning_rate_fn,
            block_size=training_args.block_size,
            beta1=training_args.beta1,
            beta2=training_args.beta2,
            diagonal_epsilon=1e-10,
            matrix_epsilon=1e-6,
            weight_decay=training_args.weight_decay,
            start_preconditioning_step=max(
                training_args.preconditioning_compute_steps + 1, 101
            ),
            preconditioning_compute_steps=training_args.preconditioning_compute_steps,
            statistics_compute_steps=1,
            best_effort_shape_interpretation=True,
            graft_type=graft_type,
            nesterov=training_args.nesterov,
            exponent_override=0,
            statistics_partition_spec=statistics_partition_spec,
            preconditioner_partition_spec=PartitionSpec(
                training_args.shard_shampoo_across, None, None
            )
            if training_args.shard_shampoo_across != "2d"
            else PartitionSpec(
                "mp" if training_args.mp_devices > training_args.dp_devices else "dp",
                None,
                None,
            ),
            num_devices_for_pjit=training_args.dp_devices,
            shard_optimizer_states=True,
            inverse_failure_threshold=0.1,
            moving_average_for_momentum=True,
            skip_preconditioning_dim_size_gt=training_args.skip_preconditioning_dim_size_gt,
            clip_by_scaled_gradient_norm=None,
            precision=jax.lax.Precision.HIGHEST,
            best_effort_memory_usage_reduction=training_args.optim_quantized,
        )

In [94]:
# what is this? chucked size for large layers with distributed shampoo
training_args.block_size

1024

In [95]:
training_args.preconditioning_compute_steps

10

In [96]:
opt

GradientTransformation(init=<function distributed_shampoo.<locals>._init_fns at 0x145bf23b0>, update=<function distributed_shampoo.<locals>.sharded_update_fn at 0x145bf1510>)

### optimizers for different parameter groups ->  `optimizer` {} and `opt_fn` {}

* __optimizer {}__ contains optimizers, i.e. GradientTransformation, which you can use to create initial optimizer state (__init__ function) , or update optimizer state (__update__ function)
* __opt_fn {}__ contains 
  * __pspec_fn__: function to create partitioan specs for optimizer state 
  * shape_and_dtype_fn: not sure what's this for yet 

In [97]:
update_fn = opt.update
optimizer = {}
opt_fn = {}

__split_params?__

what does this do? `split_params(trainable_params_shape)` 

it split the parameters into 3 groups: "standard", "scanned_encoder", "scanned_decoder"

 * FlaxBartEncoderLayers -> scanned_encoder
 * FlaxBartDecoderLayers -> scanned_decoder
 * everything else -> standard
 
the result is a dict contain all 3 parameter groups, you can access with keys standard, scanned_encoder and scanned_decoder; we then create optimizers for each parameter group

In [98]:
def split_params(data):
    """Split params between scanned and non-scanned"""
    flat = traverse_util.flatten_dict(unfreeze(data))
    split = {"standard": {}, "scanned_encoder": {}, "scanned_decoder": {}}
    for k, v in flat.items():
        if "FlaxBartEncoderLayers" in k:
            split["scanned_encoder"][k] = v
        elif "FlaxBartDecoderLayers" in k:
            split["scanned_decoder"][k] = v
        else:
            split["standard"][k] = v
    # remove empty keys
    split = {k: v for k, v in split.items() if v}
    for k, v in split.items():
        split[k] = freeze(traverse_util.unflatten_dict(v))
    return split

In [99]:
split_params(trainable_params_shape)

{'standard': FrozenDict({
     lm_head: {
         kernel: ShapeDtypeStruct(shape=(1024, 16401), dtype=float32),
     },
     model: {
         decoder: {
             embed_positions: {
                 embedding: ShapeDtypeStruct(shape=(256, 1024), dtype=float32),
             },
             embed_tokens: {
                 embedding: ShapeDtypeStruct(shape=(16401, 1024), dtype=float32),
             },
             final_ln: {
                 bias: ShapeDtypeStruct(shape=(1024,), dtype=float32),
             },
             layernorm_embedding: {
                 bias: ShapeDtypeStruct(shape=(1024,), dtype=float32),
                 scale: ShapeDtypeStruct(shape=(1024,), dtype=float32),
             },
             layers: {
                 FlaxBartDecoderLayer_0: {
                     FlaxBartAttention_0: {
                         k_proj: {
                             kernel: ShapeDtypeStruct(shape=(1024, 1024), dtype=float32),
                         },
                    

In general, optimizer is a __GradientTransformation__ object, it contains <u>a pair of functions</u>, __init__ and __update__

for __distributed_shampoo__, its __init__ function (__opt.init__) has same signature, i.e. __opt.init(params)-> opt_state__, but needs to be used differently, depends on using it in pjit or pmap mode, you need to get the appropriate init function from __opt.init(params).init__ (instead of __opt.init__ directly)
* so, you will need to first use __opt.init(params)__ to create an optimizer state object, which contains the __appropriate init__ function that you can use to create actual initial opt_state 
* the init function for pjit is __sharded_init_fn__
* __<font color=red>how to use this pjit init function??</font>__
  * it has same signature: __init(params) -> initial opt_state__
  * __input__ is __params__ , however, for scanned layers, it expects the parameters without the additional dimension, so you need either remove the dimension from data or __vmap the init function__
  * __output__ is the __initial opt state__ (__ShampooState__)
  * for __example__, this is how we find out the shape of initial optimizer state with jax.eval_shape
```python
opt_state_shape = {}
for k, p in split_params(trainable_params_shape).items():
    if "scanned" not in k:
        opt_state_shape[k] = jax.eval_shape(optimizer[k].init, p)
    else:
        opt_state_shape[k] = jax.eval_shape(jax.vmap(optimizer[k].init), p)
```
  


all 3 optimizers has same __update__ function; but the __init__ function is created seperately for different parameter groups in dalle-mini's code
   * i.e. for each parameter group __p__ ->  __opt.init(p).init_fn__
   * for __parameters in scanned layers__ , it <u>remove the first dimension</u> before we apply __opt.init()__ on it - I don't think this step is needed, the parameters are unused in the function, but makes it more readable I guess


In [100]:
for k, p in split_params(trainable_params_shape).items():
            if "scanned" in k:
            # for scanned_encoder and scanned_decoder, remove the first dimension from shapes
                p = jax.eval_shape(
                    lambda x: jax.tree_util.tree_map(lambda y: y[0], x), p
                )
            # opt.init(parameters) -> 
            optimizer[k] = opt.init(p)
            opt_fn[k] = NamedTuple("opt_fn", pspec_fn=Any, shape_and_dtype_fn=Any)(
                optimizer[k].pspec_fn, optimizer[k].shape_and_dtype_fn
            )
            optimizer[k] = optax.GradientTransformation(optimizer[k].init_fn, update_fn)

How __optimizer{}__ and __opt_fn{}__ dict is created 
```python
                                            opt.update
                                                ⬇
p->opt.init()-> opt_state ->init_fn -> GradientTransformation -> optimizer{}
                     ⬇            
              (pspec_fn, shape_and_dtype_fn)
                     ⬇
                   opt_fn{}
 ```
                                     

In [ ]:
# an example
# take the standard/unscanned parameter group as example
p = split_params(trainable_params_shape)['standard']

In [ ]:
# first step is to use shampoo.init to get an InitFnState object, 
# that contains the appropriate init functions that we can use to create initial opt state
opt.init??

In [ ]:
opt.init

In [ ]:
# InitFnState
opt.init(p)

In [ ]:
# this is the init function we need to create the initial optimizer state (ShampooState)
opt.init(p).init_fn

In [ ]:
# didn't work - because p here is not actually parameter - it is the shapes of parameters
# opt.init(p).init_fn(p)

In [ ]:
# however this will work, more on eval_shape in next section
# jax.eval_shape(opt.init(p).init_fn, p)

In [ ]:
# all 3 GrandientTransformation object actually have same init and update function? same?
id(optimizer['standard'].init),id(optimizer['scanned_encoder'].init),id(optimizer['scanned_decoder'].init)

In [ ]:
# optimizer is a dict of gradienttransformation
optimizer

In [ ]:
# opt_fn is a dict of opt_fn
opt_fn

In [ ]:
# testing
# it really seems like the parameters are not actually used 

optimizer_test = {}
opt_fn_test = {}

for k, p in split_params(trainable_params_shape).items():
            # opt.init(parameters) -> 
            optimizer_test[k] = opt.init(p)
            opt_fn_test[k] = NamedTuple("opt_fn", pspec_fn=Any, shape_and_dtype_fn=Any)(
                optimizer_test[k].pspec_fn, optimizer_test[k].shape_and_dtype_fn
            )
            optimizer_test[k] = optax.GradientTransformation(optimizer_test[k].init_fn, update_fn)

In [ ]:
optimizer_test

### about jax.eval_shape and ShapeDtypeStruct


let's start to understand __jax.eval_shape__ first, it is a utility function for performing shape inference

```python
jax.eval_shape(fun, *args, **kwargs)
```

* the __purpose of jax.eval_shape__ is to __infer__ the __shape__ of <u>fun(*args, **kwargs)</u>, without actually running the function; instead of passing the actual inputs (which should be a array, scalar or nested pytree of these types), you should use __duck-typed__ scalars and arrays intead (e.g. __ShapeDtypeStruct__), which has 2 attributes: __"shape"__, __"dtype"__ 
```python
class ShapeDtypeStruct:
  __slots__ = ["shape", "dtype"]
  def __init__(self, shape, dtype):
    self.shape = shape
    self.dtype = dtype
```
* the input of jax_eval_shape should be a __pytree of ShapeDtypeStruct__ and the output should also be a __pytree of ShapeDtypeStruct__; the behavior should be defined as 
```python
def eval_shape(fun, *args, **kwargs):
  out = fun(*args, **kwargs)
  return jax.tree_map(shape_dtype_struct, out)
```



### about set_partition

__How to use set_partition?__
* it's used in dalle-mini to create __PartitionSpecs for trainable parameters__, which is then used to create __PartitionSpec for optimizer state__ ( we will go over in next section)


```
set_partitions(trainable_params_shape) -> trainable_params_spec
```
* both __input__ (e.g.trainable_params_shape) and __output__ (e.g.trainable_params_spec)are __pytree with same structure__ (same structure as parameters tree)
* it doesn't matter what's in the input tree (in our example, we used the tree of shapes), it will be initialized as empty object before we try to find the partition rule for it
* for __each parameter__, we loop through all the __rules__, which contains the __keywords__ , and a __PartitionSpec__ object - if the parameter name contains the rule keyward, we consider it a __match__. the parameter will be assigned the <u>PartitionSpec of the first matching rule</u>

its basic defination, a few caveats:
* if use_scan (which we did not use), we will add a None in each PartitionSpec for parameters in scanned layers 
* we will make sure all the parameters has a PartitionSpec 

```python
def set_partitions(in_dict, use_scan):
    rules = _get_partition_rules()
    replace = _replacement_rules(rules)
    initd = {k: _unmatched for k in flatten_dict(in_dict)}
    result = {k: replace(k, v) for k, v in initd.items()}
    return freeze(unflatten_dict(result))
```

In [ ]:
# step by step
import re
from jax.experimental import PartitionSpec as P
from flax.traverse_util import flatten_dict, unflatten_dict

def _match(qs, ks):
    """Return True if regexes in qs match any window of strings in tuple ks."""
    # compile regexes and force complete match
    qts = tuple(map(lambda x: re.compile(x + "$"), qs))
    for i in range(len(ks) - len(qs) + 1):
        matches = [x.match(y) for x, y in zip(qts, ks[i:])]
        if matches and all(matches):
            return True
    return False

def _replacement_rules(rules):
    def replace(key, val):
        for rule, replacement in rules:
            if _match(rule, key):
                return replacement
        return val

    return replace

def _get_partition_rules():
    return [
        # embeddings
        (("embed_positions", "embedding"), P("mp", None)),
        (("embed_tokens", "embedding"), P("mp", None)),
        (("rel_bias", "embedding"), P(None, "mp")),
        # attention
        (("(q_proj|k_proj|v_proj)", "kernel"), P(None, "mp")),
        (("out_proj", "kernel"), P("mp", None)),
        # FFN
        (("Dense_0", "kernel"), P(None, "mp")),
        (("GLU.*", "Dense_1", "kernel"), P(None, "mp")),
        (("GLU.*", "Dense_2", "kernel"), P("mp", None)),
        (("FFN.*", "Dense_1", "kernel"), P("mp", None)),
        # layer norms
        (("(bias|scale)",), None),
        (("lm_head", "kernel"), P(None, "mp")),
        # head scale and tau
        (("(head_scale|tau)",), None),
    ]

rules = _get_partition_rules()
replace = _replacement_rules(rules)

In [ ]:
# each rule is a tuple 
# first element is tuple of keywords, second element is the partitionspec 
rules[0]

In [ ]:
# replace is a function that take the parameter name (here is a tuple of nested key) and the default return value
# if any of the keys matches keywards on our rule, it will return the PartitionSpec
# this example matches our first rule
replace(('model', 'decoder', 'embed_positions', 'embedding'), 0)

### get PartitionSpec for optimizer state  -> `opt_state_spec`

use __pspec_fn__ from opt.init().pspec_fn

In [101]:
    # get PartitionSpec for optimizer state
    def get_opt_state_spec_and_shape():
        # get opt_state shape without actual init
        opt_state_shape = {}
        for k, p in split_params(trainable_params_shape).items():
            if "scanned" not in k:
                opt_state_shape[k] = jax.eval_shape(optimizer[k].init, p)
            else:
                opt_state_shape[k] = jax.eval_shape(jax.vmap(optimizer[k].init), p)

        if training_args.optim == "adafactor":
            # factorized state must be replicated (rank different than params)
            opt_state_spec = {k: None for k in split_params(trainable_params_shape)}

        elif training_args.optim in ["adam", "distributed_shampoo"]:

            def _opt_state_spec_per_leaf(x, spec):
                if isinstance(x, FrozenDict):
                    # variables with same structure as params
                    return spec
                else:
                    # other variables such as count
                    return None

            split_spec = split_params(set_partitions(trainable_params_shape, False))
            opt_state_spec = {}
            for k, p in split_params(trainable_params_shape).items():
                if "scanned" in k:
                    p = jax.eval_shape(
                        lambda x: jax.tree_util.tree_map(lambda y: y[0], x), p
                    )
                if training_args.optim == "adam":
                    opt_state_spec[k] = jax.tree_util.tree_map(
                        partial(_opt_state_spec_per_leaf, spec=split_spec[k]),
                        opt_state_shape[k],
                        # return None spec for empty elements
                        is_leaf=lambda x: isinstance(x, (FrozenDict, optax.EmptyState)),
                    )
                elif training_args.optim == "distributed_shampoo":
                    opt_state_spec[k] = opt_fn[k].pspec_fn(
                        p,
                        split_spec[k],
                        statistics_partition_spec,
                    )
                # add dimension for scanned params
                if "scanned" in k:
                    opt_state_spec[k] = jax.tree_util.tree_map(
                        lambda x: PartitionSpec(*(None,) + x) if x is not None else None,
                        opt_state_spec[k],
                        is_leaf=lambda x: isinstance(x, PartitionSpec),
                    )
        else:
            raise NotImplementedError
        return freeze(opt_state_spec), freeze(opt_state_shape)

In [102]:
opt_state_spec, opt_state_shape = get_opt_state_spec_and_shape()

/Volumes/GoogleDrive/My Drive/Github/dalle-mini/tools/train/scalable_shampoo/distributed_shampoo.py:1153: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  params_flat, treedef = jax.tree_flatten(params)
/Volumes/GoogleDrive/My Drive/Github/dalle-mini/tools/train/scalable_shampoo/distributed_shampoo.py:1207: FutureWarning: jax.tree_unflatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_unflatten instead.
  local_stats = jax.tree_unflatten(treedef, local_stats_flat)
/Volumes/GoogleDrive/My Drive/Github/dalle-mini/tools/train/scalable_shampoo/distributed_shampoo.py:1267: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  param_pspec_flat, _ = jax.tree_flatten(
/Volumes/GoogleDrive/My Drive/Github/dalle-mini/tools/train/scalable_shampoo/distributed_shampoo.py:1270: FutureWarning: jax.tree_flatten is

__opt_state_shape__

it is a pytree with __same structure__ as opt_state, but contains __ShapeDtypeStruct__
* it is a dict contains one ShampooState for each parameter group
* we didn't have to actually create opt_state, we used __jax.eval_shape__ to inferred its shape


In [106]:
# we set use_scan as False, so only one parameter group
opt_state_shape

In [145]:
# all the leaves are shapes
jax.tree_leaves(opt_state_shape)

__create opt_state_spec__

1. first we create the __partition specs for parameters__ and split it into parameter groups -> __split_spec__
2. loop through the parameter groups
   * if scanned group, remove first dimension from all its parameters' shapes
   * create __opt_state_spec__ for that parameter group use __pspec_fn()__ 
     * inputs are: __the shape__ tree, __the partition specs for params__ and __partition specs for statistics__

```python
 opt_fn.pspec_fn(params_shape, params_spec, statistics_partition_spec)
```

In [151]:
opt_fn

In [149]:
# first get partition based on the params_shape and partition rules
# and then split into 3 parameter groups
split_spec = split_params(set_partitions(trainable_params_shape, False))
split_spec

In [ ]:
# standard
p = split_spec['standard']

about __pspec_fn__
  * we get __pspec_fn__ from the state return by distributed_shampoo's __init__ method (see chart below)
  * when shampoo is used in pjit mode, it is __sharded_init_partition_spec_fn__ 
```python
                                            opt.update
                                                ⬇
p -> opt.init() ->  opt_state -> init_fn -> GradientTransformation -> optimizer{}
                        ⬇            
              (pspec_fn, shape_and_dtype_fn)
                        ⬇
                      opt_fn{}
```
 __pspec_fn__ takes 3 inputs: 
   * __params__:  A pytree with params shapes
      * duck_typed is fine since we only need to use its shape info, e.g. it used trainable_params_shape 
      * its __.shape__ attribute is used, that's why we remove the first dimension of scanned parameters when we pass it to psepc_fn
   * __params_partition_spec__: A pytree with PartitionSpec for params. 
      * the split_spec we just created
   * __partition_spec_for_statistics__: PartitionSpec for the statistics. 
      *  statistics_partition_spec is created earlier when we create distributed_shampoo
  * it returns a parallel state tree with PartitionSpec associated with state??


```python
opt_state_spec[k] = opt_fn[k].pspec_fn(
                        p,
                        split_spec[k],
                        statistics_partition_spec,
                    )
```



In [152]:
opt_fn['standard'].pspec_fn

In [154]:
split_spec['standard']

In [416]:
# opt_state_spec has same tree structre as opt_state, it is also a ShampooState 
opt_state_spec['standard'].stats.local_stats

FrozenDict({
    lm_head: {
        kernel: LocalShardedParameterStats(diagonal_statistics=QuantizedValue(quantized=PartitionSpec(None, 'mp'), diagonal=[], bucket_size=[], quantized_dtype=<class 'jax.numpy.float32'>, extract_diagonal=False, shape=[1024, 16401]), diagonal_momentum=QuantizedValue(quantized=PartitionSpec(None, 'mp'), diagonal=[], bucket_size=[], quantized_dtype=<class 'jax.numpy.float32'>, extract_diagonal=False, shape=[1024, 16401]), momentum=QuantizedValue(quantized=PartitionSpec(None, 'mp'), diagonal=[], bucket_size=[], quantized_dtype=<class 'jax.numpy.float32'>, extract_diagonal=False, shape=[1024, 16401]), training_metrics=TrainingMetrics(inverse_pth_root_errors=PartitionSpec()), index_start=0, sizes=[]),
    },
    model: {
        decoder: {
            embed_positions: {
                embedding: LocalShardedParameterStats(diagonal_statistics=QuantizedValue(quantized=PartitionSpec('mp', None), diagonal=[], bucket_size=[], quantized_dtype=<class 'jax.numpy.float3

In [417]:
jax.tree_leaves(opt_state_spec['standard'].stats.global_stats)

/var/folders/9l/9hm851zx7qgfkmls0h72j82r0000gn/T/ipykernel_27605/2272418448.py:1: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  jax.tree_leaves(opt_state_spec['standard'].stats.global_stats)


[PartitionSpec(None, 'dp', None),
 PartitionSpec(None, 'dp', None),
 PartitionSpec()]

In [419]:
# all the leaves are partitionspecs
jax.tree_leaves(opt_state_spec['standard'].stats.local_stats)

/var/folders/9l/9hm851zx7qgfkmls0h72j82r0000gn/T/ipykernel_27605/2655405477.py:2: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  jax.tree_leaves(opt_state_spec['standard'].stats.local_stats)


[PartitionSpec(None, 'mp'),
 PartitionSpec(None, 'mp'),
 PartitionSpec(None, 'mp'),
 PartitionSpec(),
 PartitionSpec('mp', None),
 PartitionSpec('mp', None),
 PartitionSpec('mp', None),
 PartitionSpec(),
 PartitionSpec('mp', None),
 PartitionSpec('mp', None),
 PartitionSpec('mp', None),
 PartitionSpec(),
 PartitionSpec(),
 PartitionSpec(),
 PartitionSpec(),
 PartitionSpec(None, 'mp'),
 PartitionSpec(None, 'mp'),
 PartitionSpec(None, 'mp'),
 PartitionSpec(),
 PartitionSpec('mp', None),
 PartitionSpec('mp', None),
 PartitionSpec('mp', None),
 PartitionSpec(),
 PartitionSpec(None, 'mp'),
 PartitionSpec(None, 'mp'),
 PartitionSpec(None, 'mp'),
 PartitionSpec(),
 PartitionSpec(None, 'mp'),
 PartitionSpec(None, 'mp'),
 PartitionSpec(None, 'mp'),
 PartitionSpec(),
 PartitionSpec(None, 'mp'),
 PartitionSpec(None, 'mp'),
 PartitionSpec(None, 'mp'),
 PartitionSpec(),
 PartitionSpec('mp', None),
 PartitionSpec('mp', None),
 PartitionSpec('mp', None),
 PartitionSpec(),
 PartitionSpec(None, 'mp'),


### create a mesh

In [103]:
mesh_shape = (training_args.dp_devices, training_args.mp_devices)
devices = np.asarray(jax.devices()).reshape(*mesh_shape)
mesh = maps.Mesh(devices, ("dp", "mp"))
logger.info(f"  Mesh shape: {mesh_shape}")

In [104]:
mesh_shape

(8, 1)

In [105]:
# it's an array, with a mesh axis 'dp','mp', each element is a device, 
mesh

Mesh(array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7]]), ('dp', 'mp'))

### define TrainState

dalle-mini TrainState has custom `apply_gradient` and `create` - pay attention to how these methods are used later 

In [106]:
    class TrainState(struct.PyTreeNode):
        step: int
        params: core.FrozenDict[str, Any]
        opt_state: optax.OptState
        apply_fn: Callable = struct.field(pytree_node=False)
        tx: optax.GradientTransformation = struct.field(pytree_node=False)
        dropout_rng: jnp.ndarray = None
        epoch: int = 0
        train_time: float = 0.0  # total time the model trained
        train_samples: int = 0  # number of samples seen

        def apply_gradients(self, *, grads, **kwargs):
            grads = split_params(trainable_params(grads, training_args.embeddings_only))
            params = split_params(
                trainable_params(self.params, training_args.embeddings_only)
            )
            opt_state = {}
            # we loop over keys: "standard", "scanned_encoder", "scanned_decoder"
            for k, param in params.items():
                update_fn = self.tx[k].update
                if "scanned" in k:
                    update_fn = jax.vmap(update_fn, in_axes=(0, 0, 0), out_axes=(0, 0))
                updates, new_opt_state = update_fn(grads[k], self.opt_state[k], param)
                params[k] = optax.apply_updates(param, updates)
                opt_state[k] = new_opt_state
            params = unsplit_params(params)
            # merge with non-trainable params
            params, new_params = traverse_util.flatten_dict(
                unfreeze(self.params)
            ), traverse_util.flatten_dict(unfreeze(params))
            params.update(new_params)
            params = freeze(traverse_util.unflatten_dict(params))

            return self.replace(
                step=self.step + 1,
                params=params,
                opt_state=freeze(opt_state),
                **kwargs,
            )

        @classmethod
        def create(cls, *, apply_fn, params, tx, **kwargs):
            opt_state = {}
            for k, p in split_params(
                trainable_params(params, training_args.embeddings_only)
            ).items():
                init_fn = tx[k].init
                if "scanned" in k:
                    init_fn = jax.vmap(init_fn)
                opt_state[k] = init_fn(p)
            return cls(
                step=0,
                apply_fn=apply_fn,
                params=params,
                tx=tx,
                opt_state=freeze(opt_state),
                **kwargs,
            )

### PartitionSpec for TrainState -> `state_spec'

we are using __TrainState__ class to create the partition spec for TrainState (__state_spec__) since the structure match - we used __param_spec__  in place of params, and __opt_state_spec__ in place of opt_state

It's a good time to review all the states associated with model that we need to partition, __TrainState__ would include both the parameters and optimizer state
```python
Parameters ⇢ optimizer state ⤵
          ↪ ⇢⇢⇢⇢⇢⇢⇢⇢       TrainState
 ```

In [107]:
    # define state spec
    state_spec = TrainState(
        params=param_spec,
        opt_state=opt_state_spec,
        dropout_rng=None,
        step=None,
        epoch=None,
        train_time=None,
        train_samples=None,
        apply_fn=model.__call__,
        tx=optimizer,
    )

In [108]:
# the result is a TrainState tree with all the leaves as PartitionSpec
state_spec

TrainState(step=None, params=FrozenDict({
    lm_head: {
        kernel: PartitionSpec(None, 'mp'),
    },
    model: {
        decoder: {
            embed_positions: {
                embedding: PartitionSpec('mp', None),
            },
            embed_tokens: {
                embedding: PartitionSpec('mp', None),
            },
            final_ln: {
                bias: None,
            },
            layernorm_embedding: {
                bias: None,
                scale: None,
            },
            layers: {
                FlaxBartDecoderLayer_0: {
                    FlaxBartAttention_0: {
                        k_proj: {
                            kernel: PartitionSpec(None, 'mp'),
                        },
                        out_proj: {
                            kernel: PartitionSpec('mp', None),
                        },
                        q_proj: {
                            kernel: PartitionSpec(None, 'mp'),
                        },
         

### restore metadata 

it does not seem to do anything here 

In [109]:
attr_state = {}
keys = ["train_time", "train_samples"]
attr_state = {k: v for k, v in model_metadata.items() if k in keys}

In [110]:
attr_state

{}

## finally! create TrainState (pjit)

we use the pjitted init_state function to create TrainState

#### Refresh on default TrainState before dig ino dalle-mini TrainState

__default TrainState__

here is how the we use TrainState in training, basically use __TrainState.create()__ to create initial state, and __state.apply_gradients()__ to update the state


```python
state = TrainState.create(
          apply_fn=model.apply,
          params=variables['params'],
          tx=tx)
grad_fn = jax.grad(make_loss_fn(state.apply_fn))
for batch in data:
    grads = grad_fn(state.params, batch)
    state = state.apply_gradients(grads=grads)
```



__default optimizer__
* the __optimizer__ is __tx__ argument we passed to TrainState.create - we do not interact with optimizer directly
* it is created with __factory methods__ like `optax.adam()`, `optax.sgd()`
* it is a __GradientTransformation__ object 
  * doc here https://optax.readthedocs.io/en/latest/api.html#optax.GradientTransformation
  * it contains a pair of pure functions: __init__ and __update__
    * __init__ (TransformInitFn)
      * __input__: an example instance of the parameters whose gradients will be transformed (e.g. __params__)
      * __output__: a pytree containing the initial value for the optimizer state. (initial __opt_state__)
      * __how it's used with TrainState__: it's called during __TrainState.create()__:  `opt_state = tx.init(params)`
      
    * __update__ (TransformUpdateFn)
      * __inputs__: 
        * a pytree of candidate updates 
          * e.g. their gradient with respect to some loss (__grads__)
          * it needs to have same tree structure as the original params pytree passed to init function (same structure as params)
        * the previous __opt_state__(__opt_state__)
        * optionally, the current params. (__params__)
      * __outputs__: the computed gradient updates, and a __new opt_state__.
      * __how it's used with TrainState__: it is called during __TrainState.apply_gradients()__
        
        ```python
        updates, new_opt_state = self.tx.update(grads, self.opt_state, self.params)
        ```
        
      * update -> __optax.apply_update__ -> __new parameters__ -> new state

In [ ]:
# the method to create an adam optimizer -> GradientTransformation
type(optax.adam)

In [ ]:
# default optimizer -i.e. tx in default TrainState
optimizer_d = optax.adam(learning_rate=1e-3)

In [ ]:
optimizer_d

In [ ]:
optimizer_d.init(params)

### init_state: the function to create initial TrainState

* init_state is __pjitted__ 
```python
with mesh:
    state = pjit(
        init_state, # function
        in_axis_resources= None,
        out_axis_resources=state_spec,
        donate_argnums=(0,),
    )(params)
`````

* it wrapped around __TrainState.create()__ , and it will only have 1 input variable: __params__
* in our case, __params = None__, so init_state will first __initialize the params__ with __model.init_weights__
```python
model.init_weights(rng_key, (1,1))
```
* since we pjit this function, our result, i.e. the initial state willbe nicely sharded based on __state_spec__


we will go over all model.init_weights, dalle-mini's TrainState.create and pjit

In [111]:
def init_state(params):
    return TrainState.create(
                    apply_fn=model.__call__,
                    tx=optimizer,
                    params=maybe_init_params(params),
                    dropout_rng=dropout_rng,
                    **attr_state,
                )

In [112]:
# our params is None - so initialize it first
def maybe_init_params(params):
    if params is not None:
        # model params are correctly loaded
        return params
    else:
        # params have not been initialized yet
        return model.init_weights(model.key, model.input_shape)
initialized_params = maybe_init_params(params)

In [ ]:
with mesh:
    state = pjit(
        init_state, # function
        in_axis_resources= None,
        out_axis_resources=state_spec,
        donate_argnums=(0,),
    )(params)
del params, opt_state_spec, opt_state_shape

#### model.init_weights() -> params

we use model.init_weights() to create initial params


__about model.init_weighs__
* our model(__DalleBart__) is a cusom `FlaxBartForCOnditionalGenerationModule` from huggingface ;and __model.init_weights__ method comes with `FlaxBartForCOnditionalGenerationModule`

* usually, to create initial variable in flax, you do something like this. 

```python
variable_init = model.init(rng_key, x)
```
* __model.init_weights__ is __wrapped around__ the __model.init()__ that we'are familar with. It will __created model inputs__ for you before calls model.init(); we will look at how these inputs are created one by one 
  * input_ids, attention_mask,
  * decoder_input_ids,decoder_attention_mask,
  * position_ids,decoder_position_ids




In [ ]:
# code
model.init_weights??

__(1)input_ids__

our dummy input_ids is an array of size __1x1__ : 
* __batch_size__ is 1, i.e. the batch contain only 1 sequence; batch size doesn't matter for model.init() since the shape of parameters will be same regardless batch size 
* __sequence length__ is 1, i.e. it only contains the __eos__ token (end of sentence token)

In [171]:
# it shape 1 x 1 defined in FlaxBartPreTrainedModel (huggingface model)
model.input_shape

In [172]:
input_ids = jnp.zeros(model.input_shape, dtype="i4")

In [173]:
input_ids

In [175]:
# end of sentence token
model.config.eos_token_id

In [176]:
# for sequence, replace last token as eos 
input_ids = input_ids.at[(..., -1)].set(model.config.eos_token_id)

In [177]:
input_ids

In [178]:
input_ids.shape

about __array.at[].set()__

The __at__ property of jax numpy provides a functionally pure equivalent of in-place array modificatons.

https://jax.readthedocs.io/en/latest/_autosummary/jax.numpy.ndarray.at.html

let's see an example

In [ ]:
t1 = jnp.zeros((2,5))

In [ ]:
t1.shape

In [ ]:
test_tensor.at[(...,-1)].set(1)

__(2) attention_mask__

In [179]:
attention_mask = jnp.ones_like(input_ids)

In [180]:
attention_mask

__(3)decoder_input_ids__  & __decoder_attention_mask__

here we just set it same as input_ids 

- the actual inputs are not necessarily same, for model.init we only care about shapes

In [181]:
decoder_input_ids = input_ids

decoder_attention_mask

In [182]:
decoder_attention_mask = jnp.ones_like(input_ids)

In [183]:
decoder_attention_mask

__(4)position_ids__ is same shape as input_ids, and indicate the position of token in sequence

In [184]:
batch_size, sequence_length = input_ids.shape
position_ids = jnp.broadcast_to(jnp.arange(sequence_length)[None, :], (batch_size, sequence_length))

In [185]:
batch_size, sequence_length

In [186]:
position_ids

__(5)decoder_position_ids__, same thing, indicate the position of the tokens in decoder_position_ids

In [188]:
decoder_position_ids = jnp.broadcast_to(jnp.arange(sequence_length)[None, :], (batch_size, sequence_length))

In [189]:
decoder_position_ids

__(6)rngs__

In [190]:
rng = model.key
params_rng, dropout_rng = jax.random.split(rng)
rngs = {"params": params_rng, "dropout": dropout_rng}

In [191]:
rngs

after created all the inputs, it will call __model.init()__ -> this returns a variable dict, and we will only get "params" dict (i.e. no other intermediate variables) 

In [192]:
random_params = model.module.init(
            rngs,
            input_ids,
            attention_mask,
            decoder_input_ids,
            decoder_attention_mask,
            position_ids,
            decoder_position_ids,
        )["params"]

In [193]:
# initialized model parameters
random_params.keys()

our initial parameters! 

In [194]:
jax.tree_map(lambda x:x.shape, random_params)

#### dalle-mini TrainState.create()


In [ ]:
TrainState.create(
                    apply_fn=model.__call__,
                    tx=optimizer,
                    params=maybe_init_params(params),
                    dropout_rng=dropout_rng,
                    **attr_state,
                )

dalle-mini's `TrainState.create` method custom defined - let's take a look here

__flax TrainState.create__
```python
  @classmethod
  def create(cls, *, apply_fn, params, tx, **kwargs):
    """Creates a new instance with `step=0` and initialized `opt_state`."""
    opt_state = tx.init(params)
    return cls(
        step=0,
        apply_fn=apply_fn,
        params=params,
        tx=tx,
        opt_state=opt_state,
        **kwargs,
    )
```

__dalle-mini TrainState.create__
 ```python
  @classmethod
        def create(cls, *, apply_fn, params, tx, **kwargs):
            opt_state = {}
            for k, p in split_params(
                trainable_params(params, training_args.embeddings_only)
            ).items():
                init_fn = tx[k].init
                if "scanned" in k:
                    init_fn = jax.vmap(init_fn)
                opt_state[k] = init_fn(p)
            return cls(
                step=0,
                apply_fn=apply_fn,
                params=params,
                tx=tx,
                opt_state=freeze(opt_state),
                **kwargs,
            )
 ```

looks like the main difference is <u>how it creates __opt_state__ </u>
* __opt_state__ contains __stateful optimizer properties__ such as the current step count when using optimizer scheduels, or momemtum values)
* with the __flax default TrainState__, you do not have to worry about opt_state at all, it create the opt_state for you when you create the TrainState, and with one step <u>opt_state = tx.init(params)</u>

* how __dalle-mini__ create opt_state:
  * __tx__ is the __optimizer {}__ we create previously, it is a dict contains one GradientTransform object for each parameter group
    * <font color=red>I actually don't think each parameter group should have different optimizer</font>
  * for scanned groups, it init method expect parameter without the first dimension so we vmap the init


In [197]:
#tx
optimizer

{'standard': GradientTransformation(init=<function distributed_shampoo.<locals>.sharded_init_fn at 0x135895510>, update=<function distributed_shampoo.<locals>.sharded_update_fn at 0x14577aa70>)}

#### take a look at the state!
* params - checkout which device the parameters are located
* opt_state
* tx

In [117]:
jax.tree_map(lambda x:len(x.device_buffers), state.params)

FrozenDict({
    lm_head: {
        kernel: 8,
    },
    model: {
        decoder: {
            embed_positions: {
                embedding: 8,
            },
            embed_tokens: {
                embedding: 8,
            },
            final_ln: {
                bias: 8,
            },
            layernorm_embedding: {
                bias: 8,
                scale: 8,
            },
            layers: {
                FlaxBartDecoderLayer_0: {
                    FlaxBartAttention_0: {
                        k_proj: {
                            kernel: 8,
                        },
                        out_proj: {
                            kernel: 8,
                        },
                        q_proj: {
                            kernel: 8,
                        },
                        v_proj: {
                            kernel: 8,
                        },
                    },
                    FlaxBartAttention_1: {
                        k

In [324]:
# take one parameter as an example 
p = state.params['lm_head']['kernel']
p

ShardedDeviceArray([[ 0.00479484, -0.01371891,  0.01736489, ...,
                      0.00192936, -0.01647965,  0.0104647 ],
                    [ 0.01261756, -0.00530845,  0.020302  , ...,
                     -0.02120532, -0.01396109,  0.02460307],
                    [ 0.01544277, -0.0097948 , -0.00852894, ...,
                     -0.01192724, -0.02051834,  0.00836666],
                    ...,
                    [ 0.01032733, -0.0217177 , -0.01968484, ...,
                      0.01031603, -0.02904796,  0.03521425],
                    [-0.01267449, -0.00052427,  0.03444157, ...,
                      0.00976225, -0.03176883,  0.01764944],
                    [-0.01821935,  0.02054082,  0.00193599, ...,
                     -0.02185882,  0.01647166, -0.02641456]],                   dtype=float32)

In [329]:
# 8 devices, all duplicated arrays
len(p.device_buffers)

8

In [367]:
# the parameter is duplicated across the devices
p.device_buffers

[DeviceArray([[ 0.00479484, -0.01371891,  0.01736489, ...,  0.00192936,
               -0.01647965,  0.0104647 ],
              [ 0.01261756, -0.00530845,  0.020302  , ..., -0.02120532,
               -0.01396109,  0.02460307],
              [ 0.01544277, -0.0097948 , -0.00852894, ..., -0.01192724,
               -0.02051834,  0.00836666],
              ...,
              [ 0.01032733, -0.0217177 , -0.01968484, ...,  0.01031603,
               -0.02904796,  0.03521425],
              [-0.01267449, -0.00052427,  0.03444157, ...,  0.00976225,
               -0.03176883,  0.01764944],
              [-0.01821935,  0.02054082,  0.00193599, ..., -0.02185882,
                0.01647166, -0.02641456]], dtype=float32),
 DeviceArray([[ 0.00479484, -0.01371891,  0.01736489, ...,  0.00192936,
               -0.01647965,  0.0104647 ],
              [ 0.01261756, -0.00530845,  0.020302  , ..., -0.02120532,
               -0.01396109,  0.02460307],
              [ 0.01544277, -0.0097948 , -0.00852894

In [327]:
p.shape

(1024, 16401)

In [328]:
state_spec.params['lm_head']['kernel']

PartitionSpec(None, 'mp')

In [368]:
state.tx

{'standard': GradientTransformation(init=<function distributed_shampoo.<locals>.sharded_init_fn at 0x14a9d3520>, update=<function distributed_shampoo.<locals>.sharded_update_fn at 0x14a9d2200>)}

In [ ]:
# opt_state

In [448]:
def prod(val) : 
    res = 1 
    for ele in val: 
        res *= ele 
    return res  

In [450]:
jax.tree_leaves(jax.tree_map(lambda x: prod(x.shape), state.opt_state['standard']))

/var/folders/9l/9hm851zx7qgfkmls0h72j82r0000gn/T/ipykernel_27605/3405161829.py:1: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  jax.tree_leaves(jax.tree_map(lambda x: prod(x.shape), state.opt_state['standard']))


[1,
 981467136,
 981467136,
 936,
 16794624,
 16794624,
 16794624,
 1,
 262144,
 262144,
 262144,
 2,
 16794624,
 16794624,
 16794624,
 1,
 1024,
 1024,
 1024,
 1,
 1024,
 1024,
 1024,
 1,
 1024,
 1024,
 1024,
 1,
 1048576,
 1048576,
 1048576,
 2,
 1048576,
 1048576,
 1048576,
 2,
 1048576,
 1048576,
 1048576,
 2,
 1048576,
 1048576,
 1048576,
 2,
 1048576,
 1048576,
 1048576,
 2,
 1048576,
 1048576,
 1048576,
 2,
 1048576,
 1048576,
 1048576,
 2,
 1048576,
 1048576,
 1048576,
 2,
 2795520,
 2795520,
 2795520,
 6,
 2795520,
 2795520,
 2795520,
 6,
 2795520,
 2795520,
 2795520,
 6,
 1024,
 1024,
 1024,
 1,
 2730,
 2730,
 2730,
 3,
 1024,
 1024,
 1024,
 1,
 1024,
 1024,
 1024,
 1,
 1024,
 1024,
 1024,
 1,
 1024,
 1024,
 1024,
 1,
 1024,
 1024,
 1024,
 1,
 1024,
 1024,
 1024,
 1,
 1048576,
 1048576,
 1048576,
 2,
 1048576,
 1048576,
 1048576,
 2,
 1048576,
 1048576,
 1048576,
 2,
 1048576,
 1048576,
 1048576,
 2,
 1048576,
 1048576,
 1048576,
 2,
 1048576,
 1048576,
 1048576,
 2,
 1048576

In [460]:
jax.eval_shape(lambda x:x ,state.opt_state['standard'].stats.global_stats)

GlobalShardedParameterStats(statistics=ShapeDtypeStruct(shape=(936, 1024, 1024), dtype=float32), preconditioners=ShapeDtypeStruct(shape=(936, 1024, 1024), dtype=float32), exponents=ShapeDtypeStruct(shape=(936,), dtype=int32))

### init variables for training: local_state

In [123]:
# keep local copy of state
# basically get some state: step, epoch, train_time and train_samples, transfer them to host as numpy array
local_state = {
    k: jax.device_get(getattr(state, k)).item()
    for k in ["step", "epoch", "train_time", "train_samples"]
    }

In [124]:
# dalle-mini trainstate has some additional fields: epoch, train_time, train_samples
# all dupolicated across the dp 
state.step

ShardedDeviceArray(0, dtype=int32, weak_type=True)

In [125]:
# use device_get to transfer to host
jax.device_get(state.step)

array(0, dtype=int32)

In [126]:
local_state

{'step': 0, 'epoch': 0, 'train_time': 0.0, 'train_samples': 0}

In [127]:
# init variables for training
start_time = time.perf_counter() - local_state["train_time"]
train_metrics = None
evaluation_ran = False
save_model_ran = False
epochs = tqdm(
        range(local_state["epoch"], num_epochs),
        desc=f"Epoch ... (1/{num_epochs})",
        position=0,
        disable=jax.process_index() > 0,
    )

Epoch ... (1/3):   0%|                                                                                                                                 | 0/3 [00:00<?, ?it/s]

In [128]:
# training start
# first epoch
epoch = next(iter(epochs))

Epoch ... (1/3):   0%|                                                                                                                                 | 0/3 [00:01<?, ?it/s]


In [129]:
epoch

0

In [130]:
# update epoch in both TrainState and the local_state
state = state.replace(epoch=epoch)
local_state["epoch"] = epoch

In [131]:
# it is not a ShardedDeviceArray anymore??
state.epoch

0

### create one batch 

it could have 3 different shapes depends on if you use_vmap_trick and gradient_accumulation_steps

since our batch is a dict, we will only look at __"input_ids"__ as example
* if we don't use vmap trick and set accumulate_gradient_steps =1, we will have our batch (272, 64)
* if we use __vmap_trick__, but not accumulate gradient, we have (8, 34, 64)
* if we use __vmap_trick__ and __accumulate gradients__ for every 3 steps, we would have one more dimension for gradient accumulation, (3, 8, 34, 64)

In [499]:
node_groups = max(1, 
                  training_args.mp_devices // jax.local_device_count()
                )
loader_bs = batch_size_per_node * node_groups

In [501]:
training_args.per_device_train_batch_size

34

In [503]:
training_args.gradient_accumulation_steps

3

In [504]:
# batch_size * dp axis size * gradient accumulation 
34 * 8 * 3

816

In [500]:
batch_size_per_node

816

In [517]:
train_loader = dataset.dataloader(
                    "train",
                    loader_bs,
                    epoch,
                )

In [518]:
# get 1 batch 
batch = next(iter(train_loader))

take a look at our batch! 
* `attention_mask` and `input_ids` is output from our tokenizer of __caption__ 
* `labels` is the encoded image from vqgan (256 discrete tokens -> index of codebook)
* `decoder_input_ids` is `labels` shifted right, with bos token added in the 0 position

In [520]:
jax.eval_shape(lambda x:x, batch)

{'attention_mask': ShapeDtypeStruct(shape=(816, 64), dtype=int32),
 'decoder_input_ids': ShapeDtypeStruct(shape=(816, 256), dtype=float32),
 'input_ids': ShapeDtypeStruct(shape=(816, 64), dtype=int32),
 'labels': ShapeDtypeStruct(shape=(816, 256), dtype=int32)}

In [522]:
batch['labels']

DeviceArray([[ 1139,  8447,  1989, ...,  1268,  9035,  7771],
             [ 1270,  9889,  7580, ...,  1400,  5848,  2922],
             [12167,  6598,   217, ...,  2881,  4644, 15282],
             ...,
             [ 5098,  4159,  1810, ...,  7491, 10627,   882],
             [   23,  1556,  5360, ...,  5772, 12145,  9150],
             [ 9296,  3029, 14899, ..., 10295,  7696, 11716]],            dtype=int32)

In [523]:
batch['decoder_input_ids']

DeviceArray([[16384.,  1139.,  8447., ..., 13440.,  1268.,  9035.],
             [16384.,  1270.,  9889., ...,  4817.,  1400.,  5848.],
             [16384., 12167.,  6598., ...,  6882.,  2881.,  4644.],
             ...,
             [16384.,  5098.,  4159., ..., 11342.,  7491., 10627.],
             [16384.,    23.,  1556., ...,  8447.,  5772., 12145.],
             [16384.,  9296.,  3029., ...,  7191., 10295.,  7696.]],            dtype=float32)

In [527]:
# set correct batch size for pjit
# this is based on our base config
# i.e use_vmap_trick and accumulate gradients for 3 steps
bs_shape = (jax.local_device_count()  // training_args.mp_devices,  # local dp devices
                            training_args.per_device_train_batch_size,
                        )
if training_args.gradient_accumulation_steps > 1:
                        # reshape data into (gradient_accumulation_steps, batch_per_node, ...)
                        # to avoid any data redistribution when sharding
                        bs_shape = (
                            training_args.gradient_accumulation_steps,
                        ) + bs_shape

In [529]:
# grad_step, dp, batch 
bs_shape

(3, 8, 34)

In [530]:
# shape batch - prepare for pjit
batch = jax.tree_util.tree_map(
    lambda x: x.reshape(bs_shape + x.shape[1:]),
    batch,)
 # freeze batch to pass safely to jax transforms
batch = freeze(batch)

In [532]:
jax.eval_shape(lambda x:x, batch)

FrozenDict({
    attention_mask: ShapeDtypeStruct(shape=(3, 8, 34, 64), dtype=int32),
    decoder_input_ids: ShapeDtypeStruct(shape=(3, 8, 34, 256), dtype=float32),
    input_ids: ShapeDtypeStruct(shape=(3, 8, 34, 64), dtype=int32),
    labels: ShapeDtypeStruct(shape=(3, 8, 34, 256), dtype=int32),
})

#### create batch_simple

* __batch_simple__ is a batch created <u>__without__ vmap_trick and gradient accumulation</u>, it has shape `(8*34, ...)`, where `dp=8`, `per_device_train_batch_size=34`

In [132]:
batch_size_per_node_per_grad_step

272

In [133]:
train_loader_simple = dataset.dataloader(
                    "train",
                    batch_size_per_node_per_grad_step,
                    epoch,)

In [134]:
batch_simple = next(iter(train_loader_simple))

In [135]:
jax.eval_shape(lambda x:x, batch_simple)

{'attention_mask': ShapeDtypeStruct(shape=(272, 64), dtype=int32),
 'decoder_input_ids': ShapeDtypeStruct(shape=(272, 256), dtype=float32),
 'input_ids': ShapeDtypeStruct(shape=(272, 64), dtype=int32),
 'labels': ShapeDtypeStruct(shape=(272, 256), dtype=int32)}

In [136]:
bs_shape_simple = (batch_size_per_node_per_grad_step,)
batch_simple = jax.tree_util.tree_map(
    lambda x: x.reshape(bs_shape_simple + x.shape[1:]),
    batch_simple,)
 # freeze batch to pass safely to jax transforms
batch_simple = freeze(batch_simple)

In [137]:
jax.eval_shape(lambda x:x, batch_simple)

FrozenDict({
    attention_mask: ShapeDtypeStruct(shape=(272, 64), dtype=int32),
    decoder_input_ids: ShapeDtypeStruct(shape=(272, 256), dtype=float32),
    input_ids: ShapeDtypeStruct(shape=(272, 64), dtype=int32),
    labels: ShapeDtypeStruct(shape=(272, 256), dtype=int32),
})

### create more specs: batch_spec, grad_batch_spec, grad_param_spec


__batch_spec__ and __grad_batch_spec__ is used to partition the __minibatch__ 
```python
   p_train_step = pjit(
        train_step,
        in_axis_resources=(
            state_spec,
            grad_batch_spec
            if training_args.gradient_accumulation_steps > 1
            else batch_spec,
            None,
        ),
        out_axis_resources=(state_spec, None),
        donate_argnums=(0,),
    )
  state, train_metrics = p_train_step(state, batch, train_time)
```


* __batch_spec__ is what we use to partition __minibatch__
  * e.g. our minibatch is (272, 64), with a mesh (dp=8, mp=1)
  * the __batch_spec ('dp',)__ will shard the batch dimension (272) over the 8 devices, each will have a slice __(34, 64)__
  * <u>if we use __vmap_trick__</u>, our minibatch will have shape (8, 34, 64), the bath_spec (dp,) will result each device has __(1,34,64)__
* __grad_batch_spec__ is also used to partition __minibatch__, when __gradient_accumulation_steps > 1__
  * when the gradient_accumulation_steps > 1,e.g. gradient_accumulation_steps =3, our minibatch will have an additional grad_idx dimension (3, 272, 64)
  * with a mesh (dp=8, mp=1), the grad_batch_spec __(None, 'dp')__ will shard the batch dimension (272) over the 8 devices, each will have a slice __(3, 34, 64)__
  * if we use __vmap_trick__ and __gradient accumulation__, our batch will have shape (3,8,34,64), each device will have __(3,1,34,64)__
  
__param_spec__ and __grad_param_spec__ is used to partition __gradients__
* __grad_param_spec__ is used to partition gradients (when we use vmap_trick)

```python
loss, grads = jax.vmap(
                    grad_fn, in_axes=(None, 0, None), out_axes=(0, 0)
                )(state.params, minibatch, dropout_rng)
                # ensure they are sharded correctly
                loss = with_sharding_constraint(loss, batch_spec)
                grads = with_sharding_constraint(grads, grad_param_spec)
 ```

In [138]:
batch_spec = PartitionSpec("dp")
grad_batch_spec = PartitionSpec(None, "dp")

In [139]:
# batch_spec is what we use to partition minibatch - say it is (
batch_spec

PartitionSpec('dp',)

In [140]:
grad_batch_spec

PartitionSpec(None, 'dp')

we are using the "vmap_trick", not sure what it is yet, but it is to avoid a crash when mp_devices > 1
https://wandb.ai/dalle-mini/dalle-mini/reports/JAX-pmap-vs-pjit--VmlldzoxNDg1ODA2

In [141]:
use_vmap_trick = training_args.use_vmap_trick

# make grad_param_spec for vmap
if use_vmap_trick:
    grad_param_spec = jax.tree_util.tree_map(
        lambda x: PartitionSpec(*("dp",) + (x if x is not None else (None,))),
            param_spec,
        )

In [142]:
# basically for each PartitionSpec in param_spec, add a leading dimension and shard over dp
# don't know why we are dong this yet 
grad_param_spec

FrozenDict({
    lm_head: {
        kernel: PartitionSpec('dp', None, 'mp'),
    },
    model: {
        decoder: {
            embed_positions: {
                embedding: PartitionSpec('dp', 'mp', None),
            },
            embed_tokens: {
                embedding: PartitionSpec('dp', 'mp', None),
            },
            final_ln: {
                bias: None,
            },
            layernorm_embedding: {
                bias: None,
                scale: None,
            },
            layers: {
                FlaxBartDecoderLayer_0: {
                    FlaxBartAttention_0: {
                        k_proj: {
                            kernel: PartitionSpec('dp', None, 'mp'),
                        },
                        out_proj: {
                            kernel: PartitionSpec('dp', 'mp', None),
                        },
                        q_proj: {
                            kernel: PartitionSpec('dp', None, 'mp'),
                        },
  

## !!train step

how it is pjitted

```python
   p_train_step = pjit(
        train_step,
        in_axis_resources=(
            state_spec,
            grad_batch_spec
            if training_args.gradient_accumulation_steps > 1
            else batch_spec,
            None,
        ),
        out_axis_resources=(state_spec, None),
        donate_argnums=(0,),
    )
 ```

how it is used, after pjit, we call 
```python
state, train_metrics = p_train_step(state, batch, train_time)
```



Let's see take step by step to understand how it is defined

#### create dropout_rng

In [143]:
dropout_rng, _ = jax.random.split(state.dropout_rng)

In [144]:
dropout_rng

DeviceArray([2465931498, 3679230171], dtype=uint32)

#### shard the batch 
(based on batch_spec)

use <u>with_sharding_constraint</u> to ensure data is sharded properly

In [145]:
# when training_args.gradient_accumulation_steps = 1 and use_vamp_trick is False
minibatch = batch_simple

In [147]:
jax.eval_shape(lambda x:x, minibatch)

FrozenDict({
    attention_mask: ShapeDtypeStruct(shape=(272, 64), dtype=int32),
    decoder_input_ids: ShapeDtypeStruct(shape=(272, 256), dtype=float32),
    input_ids: ShapeDtypeStruct(shape=(272, 64), dtype=int32),
    labels: ShapeDtypeStruct(shape=(272, 256), dtype=int32),
})

In [148]:
pjit_fn_with = pjit(
  lambda batch: with_sharding_constraint(minibatch, batch_spec),
  in_axis_resources=batch_spec,
  out_axis_resources=batch_spec)
with mesh:
    minibatch = pjit_fn_with(minibatch)


In [149]:
batch_spec

PartitionSpec('dp',)

In [150]:
# we shard the first dimension of batch_simple across the 8 devices on dp
# each get 34
jax.eval_shape(lambda x:x, minibatch)

FrozenDict({
    attention_mask: ShapeDtypeStruct(shape=(272, 64), dtype=int32),
    decoder_input_ids: ShapeDtypeStruct(shape=(272, 256), dtype=float32),
    input_ids: ShapeDtypeStruct(shape=(272, 64), dtype=int32),
    labels: ShapeDtypeStruct(shape=(272, 256), dtype=int32),
})

In [151]:
jax.tree_map(lambda x:x.device_buffers[0].shape, minibatch)

FrozenDict({
    attention_mask: (34, 64),
    decoder_input_ids: (34, 256),
    input_ids: (34, 64),
    labels: (34, 256),
})

In [155]:
minibatch['labels'].shape

(272, 256)

In [157]:
minibatch['labels'].device_buffers[0].shape

(34, 256)

In [166]:
# apply jnp.mean() will bring the result back to the host 
jnp.mean(minibatch['labels'], axis=0).shape

(256,)

#### model.apply

In [225]:
model_args, labels = minibatch.pop("labels")

In [226]:
labels.shape

(272, 256)

In [234]:
model_labels = jax.tree_map(lambda x: x.device_buffers[0], labels)
model_labels = jax.device_get(model_labels)

In [227]:
model_args = jax.tree_map(lambda x: x.device_buffers[0], model_args)
model_args = jax.device_get(model_args)

In [228]:
jax.tree_map(lambda x:x.shape, model_args)

FrozenDict({
    attention_mask: (34, 64),
    decoder_input_ids: (34, 256),
    input_ids: (34, 64),
})

In [229]:
model_params = jax.tree_map(lambda x: x.device_buffers[0], state.params)
model_params = jax.device_get(model_params)

In [230]:
jax.eval_shape(lambda x:x, model_params)

FrozenDict({
    lm_head: {
        kernel: ShapeDtypeStruct(shape=(1024, 16401), dtype=float32),
    },
    model: {
        decoder: {
            embed_positions: {
                embedding: ShapeDtypeStruct(shape=(256, 1024), dtype=float32),
            },
            embed_tokens: {
                embedding: ShapeDtypeStruct(shape=(16401, 1024), dtype=float32),
            },
            final_ln: {
                bias: ShapeDtypeStruct(shape=(1024,), dtype=float32),
            },
            layernorm_embedding: {
                bias: ShapeDtypeStruct(shape=(1024,), dtype=float32),
                scale: ShapeDtypeStruct(shape=(1024,), dtype=float32),
            },
            layers: {
                FlaxBartDecoderLayer_0: {
                    FlaxBartAttention_0: {
                        k_proj: {
                            kernel: ShapeDtypeStruct(shape=(1024, 1024), dtype=float32),
                        },
                        out_proj: {
                     

In [231]:
logits = state.apply_fn(**model_args, 
                        params=model_params, 
                        dropout_rng=dropout_rng, 
                        train=True
            )[0]

In [305]:
state.apply_fn

<bound method FlaxBartPreTrainedModel.__call__ of <dalle_mini.model.modeling.DalleBart object at 0x1094455a0>>

In [232]:
logits.shape

(34, 256, 16401)

In [236]:
onehot(model_labels, logits.shape[-1]).shape

(34, 256, 16401)

# model step by step

## Review what's been done so far

__How model was created__
```python
model = DalleBart(
            config,
            seed=training_args.seed_model,
            dtype=getattr(jnp, model_args.dtype),
            _do_init=False,
        )
```

In [250]:
# it was created with _do_init=False
try: 
    model.params
except ValueError as e:
    print(e)

`params` cannot be accessed from model when the model is created with `_do_init=False`. You must call `init_weights` manually and store the params outside of the model and pass it explicitly where needed.


In [247]:
model.module

FlaxBartForConditionalGenerationModule()

In [374]:
model.config

DalleBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "attention_dropout": 0.0,
  "bos_token_id": 16385,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 2730,
  "decoder_layers": 12,
  "decoder_start_token_id": 16384,
  "do_sample": true,
  "dropout": 0.0,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 2730,
  "encoder_layers": 12,
  "encoder_vocab_size": 50300,
  "eos_token_id": 16385,
  "force_ln_scale": false,
  "gradient_checkpointing": false,
  "image_length": 256,
  "image_vocab_size": 16400,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "ln_positions": "normformer",
  "ln_type": "layernorm",
  "max_length": 257,
  "max_text_length": 64,
  "min_length": 257,
  "model_type": "dallebart",
  "normalize_text": true,
  "pad_token_id": 16385,
  "scale_embedding": false,
  "sinkhorn_iters": 1,
  "tau_init": 0.05,
  "tie_word_embeddings": false,
  "transformers_version": "4.24.0.dev0",
  "use_absolute_position_embeddings":

__How was model parameters initialized?__

it was initialized __init_state__ function
```python
with mesh:
    state = pjit(
        init_state, # function
        in_axis_resources= None,
        out_axis_resources=state_spec,
        donate_argnums=(0,),
    )(params)
`````

* it wrapped around __TrainState.create()__ , the resulting function will only have 1 input variable: __params__
* we passed __params = None__, so init_state will first __initialize the params__ with __model.init_weights__
```python
model.init_weights(rng_key, (1,1))
```
* since we pjit this function, our result, i.e. the __initial state__ willbe nicely sharded based on __state_spec__; particularlly, our __parameters__ are partitioned based on __param_spec__

for this test purpose, I convert the parameter back to host -> model_params

In [240]:
jax.eval_shape(lambda x:x , model_params)

FrozenDict({
    lm_head: {
        kernel: ShapeDtypeStruct(shape=(1024, 16401), dtype=float32),
    },
    model: {
        decoder: {
            embed_positions: {
                embedding: ShapeDtypeStruct(shape=(256, 1024), dtype=float32),
            },
            embed_tokens: {
                embedding: ShapeDtypeStruct(shape=(16401, 1024), dtype=float32),
            },
            final_ln: {
                bias: ShapeDtypeStruct(shape=(1024,), dtype=float32),
            },
            layernorm_embedding: {
                bias: ShapeDtypeStruct(shape=(1024,), dtype=float32),
                scale: ShapeDtypeStruct(shape=(1024,), dtype=float32),
            },
            layers: {
                FlaxBartDecoderLayer_0: {
                    FlaxBartAttention_0: {
                        k_proj: {
                            kernel: ShapeDtypeStruct(shape=(1024, 1024), dtype=float32),
                        },
                        out_proj: {
                     

__model arguments dict__:

I took one buffer in one device 

In [238]:
jax.eval_shape(lambda x:x, model_args)

FrozenDict({
    attention_mask: ShapeDtypeStruct(shape=(34, 64), dtype=int32),
    decoder_input_ids: ShapeDtypeStruct(shape=(34, 256), dtype=float32),
    input_ids: ShapeDtypeStruct(shape=(34, 64), dtype=int32),
})

In [242]:
state.apply_fn

<bound method FlaxBartPreTrainedModel.__call__ of <dalle_mini.model.modeling.DalleBart object at 0x1094455a0>>

## model structure
```python

(custom)              (transformers/bart)            (transformers/bart)        (transformers/modeling_flax_utils)
DalleBart    ->  FlaxBartForConditionalGeneration  -> FlaxBartPreTrainedModel -> FlaxPreTrainedModel
                                                       __call__
  ⬇ module_class                         ⬇ module_class
                                   
FlaxBartForConditionalGenerationModule -> FlaxBartForConditionalGenerationModule
(custom)                                     (transformers/bart)
  __call__ 
    
  ⬇ lm_head      ⬇ model     
                                   
  Dense         FlaxBartModule  ->  FlaxBartModule
                (custom)            (transformers/bart)

```

## prepare inputs

`FlaxBartPreTrainedModel.__call__`

__model arguments__

In [251]:
model_args.keys()

frozen_dict_keys(['attention_mask', 'decoder_input_ids', 'input_ids'])

In [336]:
input_ids = model_args['input_ids']
attention_mask = model_args['attention_mask']
decoder_input_ids = model_args['decoder_input_ids']
params=model_params 
dropout_rng=dropout_rng
train=True

# defaults
decoder_attention_mask = None
position_ids = None
decoder_position_ids = None
output_attentions = None
output_hidden_states = None
return_dict = None

In [315]:
model.config.output_attentions

False

In [316]:
model.config.output_hidden_states

False

In [317]:
model.config.return_dict

True

In [337]:
output_attentions = output_attentions if output_attentions is not None else model.config.output_attentions
output_hidden_states = (
    output_hidden_states if output_hidden_states is not None else model.config.output_hidden_states
        )
return_dict = return_dict if return_dict is not None else model.config.return_dict

__Inputs for BART encoder__

captions are encoded through a BART encoder.

* __input_ids__: tokenized caption
* __attention_mask__: output of tokenizer as well, 0 indicating padding
* __position_ids__: same shape as input_ids, indicate the position of token in each sequence

In [338]:
# the only thing we need to create is position_ids
batch_size, sequence_length = input_ids.shape
position_ids = jnp.broadcast_to(jnp.arange(sequence_length)[None, :], (batch_size, sequence_length))

In [319]:
# tokenized caption (text input) 
# first sequence
input_ids[0]

array([    0,    75,  1455,    31,    11,  7315, 16612, 25641,  5072,
        7406,    11,   176, 49895,   107,   820,   125, 10795,     2,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1], dtype=int32)

In [320]:
# attention_mask, 0 for padding positions
attention_mask[0]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

In [321]:
# position_ids indicate each token's position in the sequence 0 - 64
input_ids.shape

(34, 64)

In [323]:
position_ids[0]

DeviceArray([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,
             15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
             30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44,
             45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
             60, 61, 62, 63], dtype=int32)

__Inputs for BART decoder__

The __output of the BART encoder__ and __encoded images__ are fed through the __BART decoder__

* __decoder_input_ids__:(same as "labels" (encoded image from vqgan encoder), but shift right 1 position, adding bos token)
    * e.g. image -> tok1, tok2, tok3 -> bos, tok1, tok2
* __decoder_attention_mask__: same shape as decoder_input_ids, but all 1
* __decoder_position_ids__: same shape as decoder_input_ids, indicate position of the token in the sequence

In [339]:
# decoder_attention_mask
if decoder_attention_mask is None:
    decoder_attention_mask = jnp.ones_like(decoder_input_ids)

In [340]:
# decoder_position_ids
if decoder_position_ids is None:
            batch_size, sequence_length = decoder_input_ids.shape
            decoder_position_ids = jnp.broadcast_to(
                jnp.arange(sequence_length)[None, :], (batch_size, sequence_length)
            )

In [341]:
decoder_input_ids.shape

(34, 256)

In [325]:
decoder_input_ids[0][:10]

array([16384.,  1304., 15525., 14164.,  7191.,  8173.,  1495., 16147.,
        8173.,  2428.], dtype=float32)

In [289]:
decoder_attention_mask.shape

(34, 256)

In [292]:
decoder_position_ids[0][:10]

DeviceArray([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

from here, it calls

```python
self.module.apply(
            {"params": params or self.params},
            input_ids=jnp.array(input_ids, dtype="i4"),
            attention_mask=jnp.array(attention_mask, dtype="i4"),
            position_ids=jnp.array(position_ids, dtype="i4"),
            decoder_input_ids=jnp.array(decoder_input_ids, dtype="i4"),
            decoder_attention_mask=jnp.array(decoder_attention_mask, dtype="i4"),
            decoder_position_ids=jnp.array(decoder_position_ids, dtype="i4"),
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            deterministic=not train,
            rngs=rngs,
        )
```

where `self.module` is a dalle-mini custom  `FlaxBartForCOnditionalGenerationModule`

In [342]:
rngs = {"dropout": dropout_rng} if dropout_rng is not None else {}
rngs

{'dropout': (DeviceArray([2465931498, 3679230171], dtype=uint32),)}

In [343]:
params = {"params": model_params}
input_ids=jnp.array(input_ids, dtype="i4")
attention_mask=jnp.array(attention_mask, dtype="i4")
position_ids=jnp.array(position_ids, dtype="i4")
decoder_input_ids=jnp.array(decoder_input_ids, dtype="i4")
decoder_attention_mask=jnp.array(decoder_attention_mask, dtype="i4")
decoder_position_ids=jnp.array(decoder_position_ids, dtype="i4")
deterministic=not train

In [344]:
deterministic

False

## self.module
__custom FlaxBartForConditionalGenerationModule__


* this module is the main interface for DalleBart, it defines the layers here
  * lm_head (Dense)
  * model (FlaxBartModule)
* define `__call__` method
```python
outputs = self.model(...)
hidden_states = outputs[0]
lm_logits = self.lm_head(hidden_states)
return FlaxSeq2SeqLMOutput(logits=lm_logits, ...)
```

inputs -> FlatBartModule -> outputs[0] -> lm_head -> logits? (this is what we take to compare against encoded images to calculate loss)

### "model": FlaxBartModule
* it is a __FlaxBartModule__ (dalle-mini custom)
* defined custom __encoder__ and __decoder__ methods
  * ("model","encoder") -> __FlaxBartEncoder__
  * ("model","decoder") -> __FlaxBartDecoder__
* create embeddings for encoder and decoder
  * encoder_embed_tokens
  * decoder_embded_tokens
* inherit the `__call__` method from the FlaxBartModule (transformer/bart)
```python
encoder_outputs = self.encoder()
decoder_outputs = self.decoder(
            input_ids=decoder_input_ids,
            attention_mask=decoder_attention_mask,
            position_ids=decoder_position_ids,
            encoder_hidden_states=encoder_outputs[0],
            encoder_attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            deterministic=deterministic,
        )
return FlaxSeq2SeqModelOutput(
            last_hidden_state=decoder_outputs.last_hidden_state,
            decoder_hidden_states=decoder_outputs.hidden_states,
            decoder_attentions=decoder_outputs.attentions,
            cross_attentions=decoder_outputs.cross_attentions,
            encoder_last_hidden_state=encoder_outputs.last_hidden_state,
            encoder_hidden_states=encoder_outputs.hidden_states,
            encoder_attentions=encoder_outputs.attentions,
        )
```

create encoder embedding: __("model","encoder","embed_tokens")__

In [401]:
# vocab size for tokenizer (text)
config.encoder_vocab_size

50300

In [402]:
config.d_model

1024

In [403]:
encoder_embed_tokens = nn.Embed(
            config.encoder_vocab_size,
            config.d_model,
            embedding_init=jax.nn.initializers.normal(config.init_std),
        )


create decoder embedding

In [404]:
encoder_embed_tokens

Embed(
    # attributes
    num_embeddings = 50300
    features = 1024
    dtype = None
    param_dtype = float32
    embedding_init = init
    embedding = None
)

In [405]:
# vocab size for image tokens (vqgan codebook size)
config.image_vocab_size

16400

In [406]:
decoder_embed_tokens = nn.Embed(
    config.image_vocab_size + 1,  # image vocab size + 1 for BOS
    config.d_model,
    embedding_init=jax.nn.initializers.normal(config.init_std),
        )

In [407]:
decoder_embed_tokens

Embed(
    # attributes
    num_embeddings = 16401
    features = 1024
    dtype = None
    param_dtype = float32
    embedding_init = init
    embedding = None
)

#### define FlaxBartEncoder and FlaxBartDecoder

In [412]:
class FlaxBartEncoder(nn.Module):
    config: DalleBartConfig
    embed_tokens: nn.Embed
    dtype: jnp.dtype = jnp.float32  # the dtype of the computation
    """
    Edits:
    - offset set to 0 (no padding token)
    - use max_text_length instead of max_position_embeddings
    - use custom FlaxBartEncoderLayerCollection
    - embed_tokens cannot be None (issue at compile time)
    """

    def setup(self):
        self.dropout_layer = nn.Dropout(rate=self.config.dropout)

        embed_dim = self.config.d_model
        self.padding_idx = self.config.pad_token_id
        self.embed_scale = math.sqrt(embed_dim) if self.config.scale_embedding else 1.0

        # Bart is set up so that if padding_idx is specified then offset the embedding ids by 2
        # and adjust num_embeddings appropriately. Other models don't have this hack
        self.offset = 0
        if self.config.use_absolute_position_embeddings:
            self.embed_positions = nn.Embed(
                self.config.max_text_length + self.offset,  # image length for BOS
                embed_dim,
                embedding_init=jax.nn.initializers.normal(self.config.init_std),
            )
        self.layers = FlaxBartEncoderLayerCollection(self.config, self.dtype)
        self.layernorm_embedding = norm(
            self.config.ln_type, dtype=self.dtype, epsilon=1e-05
        )

        # postln is already applied in every layer
        if self.config.use_final_ln_encoder and self.config.ln_positions != "postln":
            self.final_ln = norm(
                self.config.ln_type,
                dtype=self.dtype,
                epsilon=1e-05,
                use_scale=self.config.force_ln_scale,
            )
        else:
            self.final_ln = None

    def __call__(
        self,
        input_ids,
        attention_mask,
        position_ids,
        output_attentions: bool = False,
        output_hidden_states: bool = False,
        return_dict: bool = True,
        deterministic: bool = True,
    ):
        input_shape = input_ids.shape
        input_ids = input_ids.reshape(-1, input_shape[-1])

        hidden_states = self.embed_tokens(input_ids) * self.embed_scale

        if self.config.use_absolute_position_embeddings:
            embed_pos = self.embed_positions(position_ids + self.offset)
            hidden_states = hidden_states + embed_pos

        hidden_states = self.layernorm_embedding(hidden_states)
        hidden_states = self.dropout_layer(hidden_states, deterministic=deterministic)

        outputs = self.layers(
            hidden_states,
            attention_mask,
            deterministic=deterministic,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        if self.final_ln is None:
            final_output = outputs[0]
        else:
            final_output = self.final_ln(outputs[0])

        if not return_dict:
            return (final_output,) + outputs[1:]

        return FlaxBaseModelOutput(
            last_hidden_state=final_output,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

class FlaxBartDecoder(nn.Module):
    config: DalleBartConfig
    embed_tokens: nn.Embed
    dtype: jnp.dtype = jnp.float32  # the dtype of the computation
    """
    Edits:
    - offset set to 0 (no padding token)
    - use image_length instead of max_position_embeddings
    - use custom FlaxBartDecoderLayerCollection
    - embed_tokens cannot be None (issue at compile time)
    """

    def setup(self):
        self.dropout_layer = nn.Dropout(rate=self.config.dropout)

        embed_dim = self.config.d_model
        self.padding_idx = self.config.pad_token_id
        self.embed_scale = (
            math.sqrt(self.config.d_model) if self.config.scale_embedding else 1.0
        )

        # Bart is set up so that if padding_idx is specified then offset the embedding ids by 2
        # and adjust num_embeddings appropriately. Other models don't have this hack
        self.offset = 0
        if self.config.use_absolute_position_embeddings:
            self.embed_positions = nn.Embed(
                self.config.image_length + self.offset,  # image length for BOS
                embed_dim,
                embedding_init=jax.nn.initializers.normal(self.config.init_std),
            )

        self.layers = FlaxBartDecoderLayerCollection(self.config, self.dtype)
        self.layernorm_embedding = norm(
            self.config.ln_type, dtype=self.dtype, epsilon=1e-05
        )

        # postln is already applied in every layer
        if self.config.use_final_ln_decoder and self.config.ln_positions != "postln":
            self.final_ln = norm(
                self.config.ln_type,
                dtype=self.dtype,
                epsilon=1e-05,
                use_scale=self.config.force_ln_scale,
            )

    def __call__(
        self,
        input_ids,
        attention_mask,
        position_ids,
        encoder_hidden_states: Optional[jnp.ndarray] = None,
        encoder_attention_mask: Optional[jnp.ndarray] = None,
        init_cache: bool = False,
        output_attentions: bool = False,
        output_hidden_states: bool = False,
        return_dict: bool = True,
        deterministic: bool = True,
    ):
        input_shape = input_ids.shape
        input_ids = input_ids.reshape(-1, input_shape[-1])

        hidden_states = self.embed_tokens(input_ids) * self.embed_scale

        if self.config.use_absolute_position_embeddings:
            embed_pos = self.embed_positions(position_ids + self.offset)
            hidden_states = hidden_states + embed_pos

        hidden_states = self.layernorm_embedding(hidden_states)
        hidden_states = self.dropout_layer(hidden_states, deterministic=deterministic)

        outputs = self.layers(
            hidden_states,
            attention_mask,
            encoder_hidden_states,
            encoder_attention_mask,
            deterministic=deterministic,
            init_cache=init_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        if self.final_ln is None:
            final_output = outputs[0]
        else:
            final_output = self.final_ln(outputs[0])

        if not return_dict:
            return (final_output,) + outputs[1:]

        return FlaxBaseModelOutputWithPastAndCrossAttentions(
            last_hidden_state=final_output,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
            cross_attentions=outputs.cross_attentions,
        )


#### ("model","encoder")

FlaxBartEncoder (dalle-mini custom)

__captions__ are encoded through a BART encoder.


It is <u>defined in FlaxBartModule </u>

create an __encoder__ now 

In [413]:
encoder = FlaxBartEncoder(
    config, dtype=model.dtype, embed_tokens=encoder_embed_tokens
        )

walk through its` __call__` method step by step

__input_ids__ contains the tokenized captions, 

first we make sure it has the correct shape __(batch_size, sequence_length)__

In [345]:
input_ids.shape

(34, 64)

In [346]:
# reshape input_ids, so it will have 2 dimensions: batch_size, sequence length
input_shape = input_ids.shape
input_ids = input_ids.reshape(-1, input_shape[-1])

input_ids.shape

apply __encoder embedding__ and __position embedding layers__ 

```python

     input_ids -> encoder.embed_tokens -> hidden_states (34,64,1024) 
                                                 ⬇️
                                              N-grammer layer
                                                 ⬇️
                                        n-gram augmented embedding
                                                 ➕                   
nput_positions -> encoder.embed_positions -> embed_pos (34,64,1024)
                                                 ⬇️
                                          hidden_states (34,64,1024)

```


In [422]:
# get the parameters
params_encoder_embed_tokens = model_params['model']['encoder']['embed_tokens']

In [424]:
params_encoder_embed_tokens['embedding'].shape

(50300, 1024)

In [425]:
hidden_states = encoder.embed_tokens.apply(
    {'params': params_encoder_embed_tokens},
    input_ids,
     rngs= None)

n-grammer

In [460]:
1024//16

64

* hidden_states here is our input to n-grammer
* for each token, its embedding has shape (1024,), and they are from an embedding size (50300,1024)
* learn a cookbook (k, h, 1024//h), where h is number of head, e.g h=16, and the codekbook size (k, 16, 64), each head has its own codebook (k, 64)
* we can device each token embedding (1024) into 16 heads also, and we will have has 16 sequences of 64 dimensions, and map each to a code-word (64) in the codebook of its corresponding head
* we can then represent it with the codeboo kid {0,1,2,...,k}

the experiments use k = 4096, 8192

In [427]:
hidden_states.shape

(34, 64, 1024)

In [453]:
input_ids[0][1]

DeviceArray(75, dtype=int32)

In [454]:
hidden_states[0][1]

DeviceArray([0.05645201, 0.00823602, 0.00430387, ..., 0.00144471,
             0.03303868, 0.03409737], dtype=float32)

In [431]:
config.scale_embedding

False

apply a scaler to embedding 

In [433]:
# we do not scale the embedding
config.scale_embedding

False

add position embedding (64,1024)
* we have 64 positions in input_ids 
* same dimension as encoding embedding, 1024


In [434]:
config.use_absolute_position_embeddings

True

In [435]:
config.max_text_length

64

In [436]:
offset =0

In [437]:
encoder_embed_positions = nn.Embed(
                config.max_text_length + offset,  # image length for BOS
                config.d_model,
                embedding_init=jax.nn.initializers.normal(config.init_std),
            )

In [438]:
encoder_embed_positions

Embed(
    # attributes
    num_embeddings = 64
    features = 1024
    dtype = None
    param_dtype = float32
    embedding_init = init
    embedding = None
)

In [444]:
params_encoder_embed_positions = model_params['model']['encoder']['embed_positions']

In [446]:
embed_pos = encoder_embed_positions.apply(
    {'params': params_encoder_embed_positions},
    position_ids + offset,
    rngs=None)

In [447]:
embed_pos.shape

(34, 64, 1024)

In [448]:
hidden_states = hidden_states + embed_pos

In [450]:
hidden_states.shape

(34, 64, 1024)

# pjit

In [567]:
pjit_fn_without = pjit(
  lambda batch: batch,
  in_axis_resources=batch_spec,
  out_axis_resources=batch_spec)

In [577]:
with mesh:
    minibatch_without = pjit_fn_without(minibatch)

In [580]:
# this is the pjit without the with_sharding_constraint - looks identical 
minibatch_without['labels'].device_buffers

[DeviceArray([[[ 1139,  8447,  1989, ...,  1268,  9035,  7771],
               [ 1270,  9889,  7580, ...,  1400,  5848,  2922],
               [12167,  6598,   217, ...,  2881,  4644, 15282],
               ...,
               [ 9035,  7266, 13586, ...,  2524,  5053, 11993],
               [ 3272,  7106, 11228, ..., 11012,   882,  9479],
               [ 5171,  5171,  5171, ..., 11096,  1400,  7738]]],            dtype=int32),
 DeviceArray([[[10042, 10042,  5171, ...,  5171, 10042,  9834],
               [14283,  8867,  5171, ..., 12566, 10042, 10042],
               [10042, 10042,  5171, ...,  9000,  2334,  1571],
               ...,
               [ 2987, 14420,  1183, ...,  1597, 11348, 12823],
               [ 9570, 10042, 14420, ...,  5416,  2601,  2813],
               [ 4950,  1847, 15565, ..., 15947,  4648,  7806]]],            dtype=int32),
 DeviceArray([[[ 5171,  5171,  1270, ...,  1941,  8283,  8283],
               [ 6060, 16279,  8283, ...,  9837, 11456, 11591],
          

In [579]:
minibatch['labels'].device_buffers

[DeviceArray([[[ 1139,  8447,  1989, ...,  1268,  9035,  7771],
               [ 1270,  9889,  7580, ...,  1400,  5848,  2922],
               [12167,  6598,   217, ...,  2881,  4644, 15282],
               ...,
               [ 9035,  7266, 13586, ...,  2524,  5053, 11993],
               [ 3272,  7106, 11228, ..., 11012,   882,  9479],
               [ 5171,  5171,  5171, ..., 11096,  1400,  7738]]],            dtype=int32),
 DeviceArray([[[10042, 10042,  5171, ...,  5171, 10042,  9834],
               [14283,  8867,  5171, ..., 12566, 10042, 10042],
               [10042, 10042,  5171, ...,  9000,  2334,  1571],
               ...,
               [ 2987, 14420,  1183, ...,  1597, 11348, 12823],
               [ 9570, 10042, 14420, ...,  5416,  2601,  2813],
               [ 4950,  1847, 15565, ..., 15947,  4648,  7806]]],            dtype=int32),
 DeviceArray([[[ 5171,  5171,  1270, ...,  1941,  8283,  8283],
               [ 6060, 16279,  8283, ...,  9837, 11456, 11591],
          

### compute_loss

In [677]:
# define loss
def loss_fn(logits, labels):
    loss = optax.softmax_cross_entropy(logits, onehot(labels, logits.shape[-1]))
    loss = loss.mean()
    return loss

def compute_loss(params, minibatch, dropout_rng):
    # minibatch has dim (batch_size, ...)
    minibatch, labels = minibatch.pop("labels")
    logits = state.apply_fn(
        **minibatch, params=params, dropout_rng=dropout_rng, train=True)[0]
    return loss_fn(logits, labels)

note that __state.apply_fn__ is defined as 
```python
TrainState.create( apply_fn=model.__call__, ...)
```

the `__call__` method is defined in __FlaxBartPreTrainedModel__, which we will look into later; 
it returns an __FlaxSeq2SeqLMOutput__ object, where its first item is __logits__, we used it to compare against __labels__ and calculate the __loss__

#### p_compute_loss

Let's pjit the compute_loss to see how it works

In [ ]:
p_compute_loss = pjit(
    compute_loss,
    in_axis_resources = (state_spec.params, batch_spec, None,),
    out_axis_resources = None)

with mesh:
    loss = p_compute_loss(state.params, minibatch, dropout_rng)

In [3]:
minibatch

NameError: name 'minibatch' is not defined

here we go over __each of the input__ to compute_loss and its __PartitionSpec__

__input(1): state.params__
* a tree of __ShardedDeviceArray__ 
* already sharded according to __state_spec.params__
* since we set mp_device = 1, we are not sharding model parameters, all the parameters are duplicated across all devices

In [639]:
# shapes of input: state.params
jax.tree_map(lambda x:x.shape, state.params)

FrozenDict({
    lm_head: {
        kernel: (1024, 16401),
    },
    model: {
        decoder: {
            embed_positions: {
                embedding: (256, 1024),
            },
            embed_tokens: {
                embedding: (16401, 1024),
            },
            final_ln: {
                bias: (1024,),
            },
            layernorm_embedding: {
                bias: (1024,),
                scale: (1024,),
            },
            layers: {
                FlaxBartDecoderLayer_0: {
                    FlaxBartAttention_0: {
                        k_proj: {
                            kernel: (1024, 1024),
                        },
                        out_proj: {
                            kernel: (1024, 1024),
                        },
                        q_proj: {
                            kernel: (1024, 1024),
                        },
                        v_proj: {
                            kernel: (1024, 1024),
                      

In [638]:
# in our case, our parameters are replicated in each device
# here take a look of the data in the first device - it's the entire parameter tree
jax.tree_map(lambda x: x.device_buffers[0].shape, state.params)

FrozenDict({
    lm_head: {
        kernel: (1024, 16401),
    },
    model: {
        decoder: {
            embed_positions: {
                embedding: (256, 1024),
            },
            embed_tokens: {
                embedding: (16401, 1024),
            },
            final_ln: {
                bias: (1024,),
            },
            layernorm_embedding: {
                bias: (1024,),
                scale: (1024,),
            },
            layers: {
                FlaxBartDecoderLayer_0: {
                    FlaxBartAttention_0: {
                        k_proj: {
                            kernel: (1024, 1024),
                        },
                        out_proj: {
                            kernel: (1024, 1024),
                        },
                        q_proj: {
                            kernel: (1024, 1024),
                        },
                        v_proj: {
                            kernel: (1024, 1024),
                      

In [646]:
# the PartitionSpec for parameters - only shard over mp, which is 1, so no sharding at all
state_spec.params

FrozenDict({
    lm_head: {
        kernel: PartitionSpec(None, 'mp'),
    },
    model: {
        decoder: {
            embed_positions: {
                embedding: PartitionSpec('mp', None),
            },
            embed_tokens: {
                embedding: PartitionSpec('mp', None),
            },
            final_ln: {
                bias: None,
            },
            layernorm_embedding: {
                bias: None,
                scale: None,
            },
            layers: {
                FlaxBartDecoderLayer_0: {
                    FlaxBartAttention_0: {
                        k_proj: {
                            kernel: PartitionSpec(None, 'mp'),
                        },
                        out_proj: {
                            kernel: PartitionSpec('mp', None),
                        },
                        q_proj: {
                            kernel: PartitionSpec(None, 'mp'),
                        },
                        v_proj: {
    

__input(2): minibatch__
* a tree of ShardedDeviceArray
* PartitionSpec is (dp,)
* so for each of its SharedDeviceArray, its __first dimension__ of its arrays is size __272__, it needs to be __sharded__ over the __8 devices__ in __dp__, so each will have __34__
* already sharded correctly

In [648]:
jax.tree_map(lambda x:x.shape, minibatch)

FrozenDict({
    attention_mask: (272, 64),
    decoder_input_ids: (272, 256),
    input_ids: (272, 64),
    labels: (272, 256),
})

In [649]:
# shard the first dimension (272) over dp (8 devices)
batch_spec

PartitionSpec('dp',)

In [650]:
# the slice of minibatch in first device
jax.tree_map(lambda x: x.device_buffers[0].shape, minibatch)

FrozenDict({
    attention_mask: (34, 64),
    decoder_input_ids: (34, 256),
    input_ids: (34, 64),
    labels: (34, 256),
})

__input(3)__: __dropout_rng__ is replicated across all devices

In [651]:
dropout_rng

DeviceArray([2465931498, 3679230171], dtype=uint32)

__output__: __loss__, replicated across all devices because out_axis_resources = None

In [661]:
loss

ShardedDeviceArray(9.912242, dtype=float32)

In [662]:
jax.device_get(loss)

array(9.912242, dtype=float32)

In [663]:
loss.device_buffers

[DeviceArray(9.912242, dtype=float32),
 DeviceArray(9.912242, dtype=float32),
 DeviceArray(9.912242, dtype=float32),
 DeviceArray(9.912242, dtype=float32),
 DeviceArray(9.912242, dtype=float32),
 DeviceArray(9.912242, dtype=float32),
 DeviceArray(9.912242, dtype=float32),
 DeviceArray(9.912242, dtype=float32)]

#### grad_fn() -> loss, grad 

if we don't apply vmap_trick, our __loss__ and __gradient__ can be calculated directly with __grad_fn__

__grad_fn__ are defined below, it is pretty standard:
* define a __compute_loss__ function that takes all the __inputs__(e.g. variables, batch, rng)  and returns a __loss__ 
```python
compute_loss(params, minibatch, dropout_rng) -> loss
``` 
* apply __jax.value_and_grad__ on it to get the __grad_fn__, which takes the same inputs but returns __loss and gradients__
```python
grad_fn(state.params, minibatch, dropout_rng) -> loss, grads
```

In [690]:
batch_spec

PartitionSpec('dp',)

In [691]:
grad_fn = jax.value_and_grad(compute_loss)

In [701]:
 def loss_and_grad(dropout_rng):
        # only 1 single rng per grad step, let us handle larger batch size (not sure why)
        dropout_rng, _ = jax.random.split(dropout_rng)
                # "vmap trick" does not work in multi-hosts and requires too much hbm
        loss, grads = grad_fn(state.params, minibatch, dropout_rng)
        # ensure grads are sharded
        grads = with_sharding_constraint(grads, param_spec)
            # return loss and grads
        return grads

In [ ]:
p_loss_and_grad = pjit(
    loss_and_grad,
    in_axis_resources = None,
    out_axis_resources = param_spec)

with mesh:
    grads = p_loss_and_grad(dropout_rng)

#### grad_fn (use_vmap_trick)

when use vmap trick, we vamp <u>same __grad_fn__ function</u> over same __minibatch__

In [ ]:
loss, grads = jax.vmap(
                    grad_fn, in_axes=(None, 0, None), out_axes=(0, 0)
                )(state.params, minibatch, dropout_rng)

In [588]:
jaxminibatch

FrozenDict({
    attention_mask: ShardedDeviceArray([[[1, 1, 1, ..., 0, 0, 0],
                         [1, 1, 1, ..., 0, 0, 0],
                         [1, 1, 1, ..., 0, 0, 0],
                         ...,
                         [1, 1, 1, ..., 0, 0, 0],
                         [1, 1, 1, ..., 0, 0, 0],
                         [1, 1, 0, ..., 0, 0, 0]],
    
                        [[1, 1, 1, ..., 0, 0, 0],
                         [1, 1, 1, ..., 0, 0, 0],
                         [1, 1, 1, ..., 0, 0, 0],
                         ...,
                         [1, 1, 1, ..., 0, 0, 0],
                         [1, 1, 1, ..., 0, 0, 0],
                         [1, 1, 0, ..., 0, 0, 0]],
    
                        [[1, 1, 1, ..., 0, 0, 0],
                         [1, 1, 0, ..., 0, 0, 0],
                         [1, 1, 0, ..., 0, 0, 0],
                         ...,
                         [1, 1, 0, ..., 0, 0, 0],
                         [1, 1, 1, ..., 0, 0, 0],
                   

#### accumulate gradients

In [ ]:
# accumulate gradients
def cumul_minibatch_step(grad_idx, cumul_loss_grad_dropout):
                cumul_loss, cumul_grads, dropout_rng = cumul_loss_grad_dropout
                loss, grads, dropout_rng = loss_and_grad(grad_idx, dropout_rng)
                cumul_loss, cumul_grads = jax.tree_util.tree_map(
                    jnp.add, (cumul_loss, cumul_grads), (loss, grads)
                )
                cumul_grads = with_sharding_constraint(cumul_grads, param_spec)
                return cumul_loss, cumul_grads, dropout_rng

In [ ]:
# create initial state for cumul_minibatch_step loop
init_minibatch_step = (
                0.0,
                with_sharding_constraint(
                    jax.tree_util.tree_map(jnp.zeros_like, state.params), param_spec
                ),
                state.dropout_rng,
            )

#### model 

```python
model = DalleBart(
            config,
            seed=training_args.seed_model,
            dtype=getattr(jnp, model_args.dtype),
            _do_init=False,
        )
```

In [381]:
config

DalleBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "attention_dropout": 0.0,
  "bos_token_id": 16385,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 2730,
  "decoder_layers": 12,
  "decoder_start_token_id": 16384,
  "do_sample": true,
  "dropout": 0.0,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 2730,
  "encoder_layers": 12,
  "encoder_vocab_size": 50300,
  "eos_token_id": 16385,
  "force_ln_scale": false,
  "gradient_checkpointing": false,
  "image_length": 256,
  "image_vocab_size": 16400,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "ln_positions": "normformer",
  "ln_type": "layernorm",
  "max_length": 257,
  "max_text_length": 64,
  "min_length": 257,
  "model_type": "dallebart",
  "normalize_text": true,
  "pad_token_id": 16385,
  "scale_embedding": false,
  "sinkhorn_iters": 1,
  "tau_init": 0.05,
  "tie_word_embeddings": false,
  "transformers_version": "4.24.0.dev0",
  "use_absolute_position_embeddings":

In [384]:
model.__dict__.keys()

dict_keys(['_config', '_module', 'key', 'dtype', 'input_shape', '_is_initialized', '_params_shape_tree', '_required_params'])

In [390]:
model.module

FlaxBartForConditionalGenerationModule()

In [401]:
model.dtype

jax.numpy.bfloat16

In [403]:
from transformers.models.bart.modeling_flax_bart import (
    FlaxBartAttention,
    FlaxBartForConditionalGeneration,
    FlaxBartForConditionalGenerationModule,
    FlaxBartModule,
)

In [407]:
FlaxBartModule(config=config, dtype=model.dtype)

FlaxBartModule(
    # attributes
    config = DalleBartConfig {
      "activation_dropout": 0.0,
      "activation_function": "gelu",
      "attention_dropout": 0.0,
      "bos_token_id": 16385,
      "d_model": 1024,
      "decoder_attention_heads": 16,
      "decoder_ffn_dim": 2730,
      "decoder_layers": 12,
      "decoder_start_token_id": 16384,
      "do_sample": true,
      "dropout": 0.0,
      "encoder_attention_heads": 16,
      "encoder_ffn_dim": 2730,
      "encoder_layers": 12,
      "encoder_vocab_size": 50300,
      "eos_token_id": 16385,
      "force_ln_scale": false,
      "gradient_checkpointing": false,
      "image_length": 256,
      "image_vocab_size": 16400,
      "init_std": 0.02,
      "is_encoder_decoder": true,
      "ln_positions": "normformer",
      "ln_type": "layernorm",
      "max_length": 257,
      "max_text_length": 64,
      "min_length": 257,
      "model_type": "dallebart",
      "normalize_text": true,
      "pad_token_id": 16385,
      "scale_em

In [400]:
model.module.config

DalleBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "attention_dropout": 0.0,
  "bos_token_id": 16385,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 2730,
  "decoder_layers": 12,
  "decoder_start_token_id": 16384,
  "do_sample": true,
  "dropout": 0.0,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 2730,
  "encoder_layers": 12,
  "encoder_vocab_size": 50300,
  "eos_token_id": 16385,
  "force_ln_scale": false,
  "gradient_checkpointing": false,
  "image_length": 256,
  "image_vocab_size": 16400,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "ln_positions": "normformer",
  "ln_type": "layernorm",
  "max_length": 257,
  "max_text_length": 64,
  "min_length": 257,
  "model_type": "dallebart",
  "normalize_text": true,
  "pad_token_id": 16385,
  "scale_embedding": false,
  "sinkhorn_iters": 1,
  "tau_init": 0.05,
  "tie_word_embeddings": false,
  "transformers_version": "4.24.0.dev0",
  "use_absolute_position_embeddings":

In [387]:
model.module.init??

Signature:
model.module.init(
    rngs: Union[Any, Dict[str, Any]],
    *args,
    method: Optional[Callable[..., Any]] = None,
    mutable: Union[bool, str, Collection[str], ForwardRef('DenyList')] = DenyList(deny='intermediates'),
    capture_intermediates: Union[bool, Callable[[ForwardRef('Module'), str], bool]] = False,
    **kwargs,
) -> flax.core.frozen_dict.FrozenDict[str, typing.Mapping[str, typing.Any]]
Source:   
  @traceback_util.api_boundary
  def init(self,
           rngs: Union[PRNGKey, RNGSequences],
           *args,
           method: Optional[Callable[..., Any]] = None,
           mutable: CollectionFilter = DenyList('intermediates'),
           capture_intermediates: Union[bool, Callable[['Module', str], bool]] = False,
           **kwargs) -> FrozenVariableDict:
    """Initializes a module method with variables and returns modified variables.

    Jitting `init` initializes a model lazily using only the shapes of the
    provided arguments, and avoids computing the

In [377]:
model.shared.num_embeddings

AttributeError: 'DalleBart' object has no attribute 'shared'